<a id="indice"></a> 
# Limpieza de características de inmueble: "rooms", "floor" y otras

[0) Mínima mirada y tratamiento previo del dataset](#0)

[1) Limpieza y transformación de "rooms"](#1)

$\hspace{.5cm}$[1.1) Analizo el dato "rooms" vs lo que puedo encontrar en "description"](#1.1)

$\hspace{.5cm}$$\hspace{.5cm}$[1.1.a) Inmuebles de 2 ambientes](#1.1.a)

$\hspace{.5cm}$$\hspace{.5cm}$[1.1.b) Inmuebles de 3 ambientes](#1.1.b)

$\hspace{.5cm}$$\hspace{.5cm}$[1.1.c) Inmuebles de 1 ambiente](#1.1.c)

$\hspace{.5cm}$$\hspace{.5cm}$[1.1.d) Inmuebles de 4 ambientes](#1.1.d)

$\hspace{.5cm}$$\hspace{.5cm}$[1.1.e) Dropear dataset según cantidad de ambientes](#1.1.e)

$\hspace{.5cm}$[1.2) Limpieza para cuando el regex tira un ambiente pero es una casa](#1.2)

$\hspace{.5cm}$[1.3) Imputación de valores en "ambientes_regex" y borrar "rooms"](#1.3)

[2) Construcción de otras variables que podrían influir en el valor de inmueble:](#2)

2.1) Parrilla   2.2) Cochera   2.3) Pileta   2.4) SUM   2.5) Solarium   2.6) Gym   2.7) Balcón   2.8) Jardín   2.9) Lavadero   2.10) A estrenar    2.11) Quincho   2.12) Patio   2.13) En pozo + dropear 2.14) Presenta edificio + dropear

[3) Limpieza y transformación de número de piso](#3)

<a id="0"></a>
## Mínima mirada y tratamiento previo del dataset

[Ir al índice](#indice)

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import plotly as pl
import plotly.express as px
import chart_studio
import chart_studio.plotly as py
import re

In [4]:
# configuro para que muestre todas las columnas y filas cuando quiero ver tablas
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
# cargo los datos
data_location = "Data/properatti.csv"
data = pd.read_csv(data_location)

In [6]:
data.shape

(121220, 26)

In [7]:
# dropeo duplicados en el dataset si es que los hay
data = data.drop_duplicates()

In [8]:
data.shape

(121220, 26)

In [9]:
# Para evitar problemas con los strings, convertimos todo el df en minuscula
data = data.applymap(lambda x: x.lower() if type(x) is str else x)

In [10]:
data.head(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,ph,mataderos,|argentina|capital federal|mataderos|,argentina,capital federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,usd,1093959.0,62000.0,55.0,40.0,1127.272727,1550.0,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 ambientes tipo casa planta baja por pasillo,...",2 amb tipo casa sin expensas en pb,https://thumbs4.properati.com/8/bluuyihjlhgiik...
1,1,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|,argentina,bs.as. g.b.a. zona sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,usd,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,venta de departamento en décimo piso al frente...,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpvbu2ztha7jv...


In [11]:
data.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [12]:
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [13]:
# miro los valores nulos
data.isnull().sum()/len(data)

Unnamed: 0                    0.000000
operation                     0.000000
property_type                 0.000000
place_name                    0.000190
place_with_parent_names       0.000000
country_name                  0.000000
state_name                    0.000000
geonames_id                   0.154405
lat-lon                       0.425260
lat                           0.425260
lon                           0.425260
price                         0.168372
currency                      0.168380
price_aprox_local_currency    0.168372
price_aprox_usd               0.168372
surface_total_in_m2           0.324435
surface_covered_in_m2         0.164222
price_usd_per_m2              0.433947
price_per_m2                  0.276869
floor                         0.934837
rooms                         0.609058
expenses                      0.882346
properati_url                 0.000000
description                   0.000016
title                         0.000000
image_thumbnail          

<a id="1"></a> 
## 1) Limpieza y transformación de "rooms"

[Ir a índice](#indice)

In [14]:
data.rooms.value_counts()

3.0     12567
2.0     12433
4.0      7937
1.0      7377
5.0      3782
6.0      1403
7.0       770
8.0       409
10.0      226
9.0       184
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
17.0       16
16.0       11
20.0       11
22.0        8
18.0        6
21.0        5
30.0        4
25.0        4
19.0        4
32.0        3
23.0        2
24.0        1
27.0        1
29.0        1
31.0        1
28.0        1
Name: rooms, dtype: int64

In [15]:
len(data)

121220

In [16]:
data.rooms.isnull().sum()

73830

In [17]:
# reemplazo nulos con 0, sabiendo que el dato 0 no existe en rooms. Para que más adelante me lo tome el value_counts o el pivot table
data["rooms"] = data.rooms.fillna(0)
data.rooms.value_counts()

0.0     73830
3.0     12567
2.0     12433
4.0      7937
1.0      7377
5.0      3782
6.0      1403
7.0       770
8.0       409
10.0      226
9.0       184
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
17.0       16
20.0       11
16.0       11
22.0        8
18.0        6
21.0        5
25.0        4
19.0        4
30.0        4
32.0        3
23.0        2
24.0        1
29.0        1
27.0        1
31.0        1
28.0        1
Name: rooms, dtype: int64

In [18]:
data.rooms.value_counts(normalize = True)
# 61% nulos aprox.

0.0     0.609058
3.0     0.103671
2.0     0.102566
4.0     0.065476
1.0     0.060856
5.0     0.031199
6.0     0.011574
7.0     0.006352
8.0     0.003374
10.0    0.001864
9.0     0.001518
11.0    0.000594
12.0    0.000536
13.0    0.000272
14.0    0.000223
15.0    0.000214
17.0    0.000132
20.0    0.000091
16.0    0.000091
22.0    0.000066
18.0    0.000049
21.0    0.000041
25.0    0.000033
19.0    0.000033
30.0    0.000033
32.0    0.000025
23.0    0.000016
24.0    0.000008
29.0    0.000008
27.0    0.000008
31.0    0.000008
28.0    0.000008
Name: rooms, dtype: float64

In [21]:
# no sé por qué, pero funciona y con esto veo los gráficos
import plotly.io as pio
pio.renderers.default='iframe'

In [79]:
data['contador'] = 1
cantidad_rooms = data.pivot_table(index=["rooms"],values=['contador'], aggfunc=len) 
cantidad_rooms = cantidad_rooms.stack(level=[0],dropna=False).to_frame('Cantidad').reset_index() #esto funciona, no termino de entender por qué, aprenderlo

# ploty gráfico
fig = px.bar(cantidad_rooms, x="rooms", y="Cantidad",
             labels={'rooms':'Cantidad de ambientes', 'Cantidad':'Total de inmuebles'},
             title="Distribución de cantidad de ambientes por inmueble (0 es valor nulo)")
fig.show()
data = data.drop(columns = ['contador'])

In [94]:
data['contador'] = 1
cantidad_rooms_property = data.pivot_table(index=["rooms","property_type"],values=['contador'], aggfunc=len) 
cantidad_rooms_property = cantidad_rooms_property.stack(level=[0],dropna=False).to_frame('Cantidad').reset_index() #esto funciona, no termino de entender por qué, aprenderlo
cantidad_rooms_property_6 = cantidad_rooms_property[(cantidad_rooms_property["rooms"]<=6) & (cantidad_rooms_property["rooms"]!=0)] #filtro hasta 6 ambientes y saco los nulos

# ploty gráfico
fig = px.bar(cantidad_rooms_property_6, x="rooms", y="Cantidad",
             color='property_type', barmode='group',
             height=400,
             labels={'rooms':'Cantidad de ambientes', 'Cantidad':'Total de inmuebles', 'property_type':'Tipo de inmueble'},
             title="Distribución de cantidad de ambientes (entre 1 y 6) por inmueble")
fig.show()
data = data.drop(columns = ['contador'])

In [95]:
data.rooms.quantile

count    121220.000000
mean          1.204430
std           1.900956
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max          32.000000
Name: rooms, dtype: float64

In [97]:
# ¿Puedo verlo por property_type en columnas?
# data.rooms.describe()
# Seguir corrigiendo

data['contador'] = 1
cantidad_rooms_property = data.pivot_table(index=["rooms","property_type"],values=['contador'], aggfunc=len) 
cantidad_rooms_property = cantidad_rooms_property.stack(level=[0],dropna=False).to_frame('Cantidad').reset_index()
cantidad_rooms_property = cantidad_rooms_property.pivot_table(index=["property_type"],values=['rooms'], aggfunc=[np.mean,np.std,np.min,np.max]) 
cantidad_rooms_property

,mean,std,amin,amax
,rooms,rooms,rooms,rooms
property_type,,,,
apartment,11.956522,8.363765,0.0,32.0
house,15.687500,9.653472,0.0,32.0
ph,7.400000,5.329165,0.0,20.0
store,4.600000,3.204164,0.0,10.0


In [179]:
# intentar hacer un describe de rooms por tipo de propiedad. Tanto en tabla como en boxplot

In [180]:
# tabla

In [181]:
# boxplot con sns o plotly

In [182]:
# barchart con plotly

<a id="1.1"></a> 
#### 1.1) Analizo el dato "rooms" vs lo que puedo encontrar en "description"

[Ir a índice](#indice)

In [183]:
# veo comentarios con todos sus caracteres, los voy a usar para preparar el patróx regex
comentrooms = data.sample(7)
list(comentrooms.description)

['muy buena casa, en duplex. 3 ambientes, living, cocina con comedor de diario, lavadero. posee dos dormitorios con posibilidad de un 3ro.  muy luminoso, pulmón de manzana.excelente ubicación, en muy buen barrio cerca de edison.carlos a. bezruk mat. 3275 cmsi  cucicba  mat. 5952  carlos a. bezruk mat. 3275 cmsicucicba  mat. 5952 carlos bezruk inmueblesav. fleming 2800 martinez y rotativas',
 'hermoso monoambiente, ideal profesional!!!apartamentos del campus tiene su ingreso por la ruta 234 av. teniente general juan domingo perón.al ingresar nos encontramos con un living-comedor , cocina separada y baño.características generales:superficie total y cubierta del departamento: 30 m2superficie total y cubierta de cochera y baulera 16m2ubicación: frentedpto. con mejoras importantes!! el mismo posee una comoda cocina , y baño ampliado con ducha y mampara.division con mueble para dormitorio y living comedor , amplio placard con puertas espejadas que le da amplitud.cielorraso con luces dicroica

In [184]:
# por ahora es un string lo que va a ser regex
patron_ambientescompleto = ".?(?P<cantidad_ambientes>\d\d?|un|uno|dos|tres|cuatro|cinco|mono)\s?\s?(?P<ambiente>amb)(?P<otra_forma>.?t?\s?)(?P<cantidad_ambientes_alternativa>\d?)"
# el compile lo que hace es pasarlo a formato pattern
regex_ambientescompleto = re.compile(patron_ambientescompleto)

In [185]:
encuentra_ambientes = data.description.apply(lambda x: None if x is np.NaN else regex_ambientescompleto.search(x))
encuentra_ambientes = encuentra_ambientes.apply(lambda x: x if x is None else x.group('cantidad_ambientes','ambiente','otra_forma','cantidad_ambientes_alternativa'))
encuentra_ambientes.value_counts()

(2, amb, i, )          10301
(3, amb, i, )          10199
(mono, amb, i, )        7382
(4, amb, i, )           5815
(dos, amb, i, )         2764
(1, amb, i, )           2269
(5, amb, i, )           1633
(tres, amb, i, )        1338
(2, amb,  , )           1161
(3, amb,  , )           1057
(un, amb, i, )          1006
(2, amb, . , )           583
(3, amb, . , )           492
(6, amb, i, )            471
(4, amb,  , )            441
(4, amb, . , )           308
(cuatro, amb, i, )       303
(1, amb,  , )            223
(7, amb, i, )            208
(5, amb,  , )            122
(1, amb, . , )           101
(2, amb,   , )            99
(3, amb, , , )            87
(2, amb, , , )            87
(5, amb, . , )            81
(3, amb, ., )             80
(2, amb, ., )             73
(3, amb,   , )            71
(cinco, amb, i, )         57
(2, amb, p, )             53
(6, amb,  , )             40
(4, amb, , , )            40
(4, amb, ., )             33
(8, amb, i, )             31
(3, amb,  t, )

In [186]:
encuentra_ambientes = data.description.apply(lambda x: None if x is np.NaN else regex_ambientescompleto.search(x))
data['ambientes_regex'] = encuentra_ambientes.apply(lambda x: 'No se encontró' if x is None else x.group('cantidad_ambientes'))

In [187]:
data['ambientes_regex'].value_counts()

No se encontró    71335
2                 12507
3                 12150
mono               7404
4                  6728
dos                2808
1                  2638
5                  1889
tres               1359
un                 1013
6                   557
cuatro              304
7                   228
cinco                57
8                    38
9                    21
00                   18
02                   16
10                   14
11                   13
72                    8
12                    6
50                    6
13                    5
23                    5
14                    5
53                    5
04                    5
33                    4
24                    4
73                    4
42                    4
93                    3
83                    3
62                    3
28                    3
26                    3
18                    3
15                    3
22                    3
06                    3
36              

In [188]:
data['ambientes_regex'].value_counts(normalize=True)

No se encontró    0.588475
2                 0.103176
3                 0.100231
mono              0.061079
4                 0.055502
dos               0.023164
1                 0.021762
5                 0.015583
tres              0.011211
un                0.008357
6                 0.004595
cuatro            0.002508
7                 0.001881
cinco             0.000470
8                 0.000313
9                 0.000173
00                0.000148
02                0.000132
10                0.000115
11                0.000107
72                0.000066
12                0.000049
50                0.000049
13                0.000041
23                0.000041
14                0.000041
53                0.000041
04                0.000041
33                0.000033
24                0.000033
73                0.000033
42                0.000033
93                0.000025
83                0.000025
62                0.000025
28                0.000025
26                0.000025
1

<h5>Veo que el regex no capta gran parte de los inmuebles (59% nulos). Sin embargo, la columna "rooms" también tenía muchos nulos (61% nulos).<br>
El siguiente trabajo consiste en chequear los que sí capta, para ver qué dice la columna "rooms" y que de alguna manera eso me verifique la consistencia de esa columna

<a id="1.1.a"></a> 
#### 1.1.a) Inmuebles de 2 ambientes

[Ir a índice](#indice)

In [189]:
# chequeo cantidades entre lo que el regex devuelve para dos ambientes y lo que dice el rooms
dos_ambientes = data[(data['ambientes_regex'] == 'dos') | (data['ambientes_regex'] == '2')]
print('Cantidades que me tira el regex')
print(dos_ambientes.ambientes_regex.value_counts())
print('')
print('Cantidades que me tira el rooms')
print(dos_ambientes.rooms.value_counts())
print('')
print('Cantidades que me tira el rooms en %')
print(dos_ambientes.rooms.value_counts(normalize=True))

Cantidades que me tira el regex
2      12507
dos     2808
Name: ambientes_regex, dtype: int64

Cantidades que me tira el rooms
0.0     7531
2.0     6907
1.0      541
3.0      180
4.0       74
5.0       45
6.0       17
10.0       6
7.0        5
9.0        2
8.0        2
12.0       2
11.0       1
13.0       1
20.0       1
Name: rooms, dtype: int64

Cantidades que me tira el rooms en %
0.0     0.491740
2.0     0.450996
1.0     0.035325
3.0     0.011753
4.0     0.004832
5.0     0.002938
6.0     0.001110
10.0    0.000392
7.0     0.000326
9.0     0.000131
8.0     0.000131
12.0    0.000131
11.0    0.000065
13.0    0.000065
20.0    0.000065
Name: rooms, dtype: float64


<h5>Voy a chequear samples de descripciones en las que difieren para entender cuál de las dos columnas es más fiable

In [190]:
# chequeo sample de los inmuebles que el regex tiró dos ambientes y el rooms dice algo distinto
aux_2_rooms = data[((data['ambientes_regex'] == 'dos') | (data['ambientes_regex'] == '2')) & (data['rooms'] != 2)].sample(5)
aux_2_rooms

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
85802,85802,sell,apartment,pinamar,|argentina|buenos aires costa atlántica|pinamar|,argentina,buenos aires costa atlántica,3429971.0,NaN,NaN,NaN,77000.0,usd,1358626.50,77000.0,40.0,40.0,1925.000000,1925.000000,NaN,0.0,NaN,http://www.properati.com.ar/1b3j1_venta_depart...,excelente departamento de 2 ambientesconsta de...,departamento en pinamar,https://thumbs4.properati.com/4/gesur6fi5r8f6n...,2
13301,13301,sell,apartment,san miguel,|argentina|bs.as. g.b.a. zona norte|san miguel|,argentina,bs.as. g.b.a. zona norte,NaN,"-34.5493377474,-58.7015435268",-34.549338,-58.701544,70000.0,usd,1235115.00,70000.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,http://www.properati.com.ar/16opx_venta_depart...,departamento 2 amb: living comedor con cocina ...,departamento 2 amb impecable apto credito iaf ...,https://thumbs4.properati.com/6/mfqtulcty6gqx7...,2
18272,18272,sell,apartment,tres de febrero,|argentina|bs.as. g.b.a. zona oeste|tres de fe...,argentina,bs.as. g.b.a. zona oeste,3427655.0,"-34.60802,-58.529177",-34.608020,-58.529177,220000.0,usd,3881790.00,220000.0,NaN,176.0,NaN,1250.000000,NaN,4.0,NaN,http://www.properati.com.ar/175rb_venta_depart...,corredor responsable: raul hector merlo - cmcp...,depto 4 amb c/ cochera + depto 2 amb- apto cre...,https://thumbs4.properati.com/7/x6u6k9z20bpwgo...,2
72204,72204,sell,apartment,palermo,|argentina|capital federal|palermo|,argentina,capital federal,3430234.0,"-34.5881354318,-58.4395152703",-34.588135,-58.439515,180000.0,usd,3176010.00,180000.0,57.0,48.0,3157.894737,3750.000000,NaN,0.0,NaN,http://www.properati.com.ar/1am01_venta_depart...,excelente departamento a estrenar en palermo h...,excelenet dos ambientes con balcon y cochera o...,https://thumbs4.properati.com/8/wbwfycgwhf2dfm...,dos
66604,66604,sell,apartment,ituzaingó,|argentina|corrientes|ituzaingó|,argentina,corrientes,3433357.0,NaN,NaN,NaN,84940.0,usd,1498723.83,84940.0,54.0,54.0,1572.962963,1572.962963,NaN,0.0,1.0,http://www.properati.com.ar/1ae21_venta_depart...,en la zona sur de la ciudad de ituzaingó a sol...,excelente departamento de 2 ambientes a estren...,https://thumbs4.properati.com/9/xhkipaclifzqfr...,2


In [191]:
list(aux_2_rooms.description)

['excelente departamento de 2 ambientesconsta de:- amplio y luminoso living comedor con estufa, pisos cerámicos y con salida al balcón.-cómodo dormitorio con placard y gran ventanal .-cocina con muebles bajo y sobre mesada y espacio para lavarropas.-baño completo.-balcón corrido.*todos los servicios',
 'departamento 2 amb: living comedor con cocina integrada - baño completo - dormitorio con placard e interior de placard. easybroker id: eb-aq6814',
 'corredor responsable: raul hector merlo - cmcpsi 5438contacto: cecilia berdeal - mls id # 61166-109apto credito. la propiedad esta constituida por 2 departamentos.  planta baja: departamento de 2 ambientes y entrada independiente. planta alta: departamento de 4 ambientes con cochera. amplio living comedor. todos los ambientes son muy luminosos. posee 2 balcones al frente. excelente ubicacion. a 3 cuadras del tren san martin. combinacion con subte b en estacion dorrego y con subte d en palermo. a 1 cuadra de lope de vega y general paz. a 5 m

<h5>Después de leer varios samples, generalmente es más fiable la columna "ambientes_regex".

In [192]:
# chequeo sample de los inmuebles que el regex tiró dos ambientes y el rooms dice dos ambientes
aux_2_rooms_coincide = data[((data['ambientes_regex'] == 'dos') | (data['ambientes_regex'] == '2')) & (data['rooms'] == 2)].sample(5)
aux_2_rooms_coincide

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
15451,15451,sell,ph,san fernando,|argentina|bs.as. g.b.a. zona norte|san fernan...,argentina,bs.as. g.b.a. zona norte,NaN,"-34.4538615,-58.543316",-34.453862,-58.543316,138000.0,usd,2434941.0,138000.0,55.0,50.0,2509.090909,2760.000000,NaN,2.0,NaN,http://www.properati.com.ar/16v0s_alquiler_ph_...,venta de departamento 1 dorm con cochera en vi...,ph en venta,https://thumbs4.properati.com/7/6wpczqukpjb20j...,2
29316,29316,sell,apartment,morón,|argentina|bs.as. g.b.a. zona oeste|morón|morón|,argentina,bs.as. g.b.a. zona oeste,3430544.0,NaN,NaN,NaN,1440000.0,usd,25408080.0,1440000.0,NaN,50.0,NaN,28800.000000,NaN,2.0,NaN,http://www.properati.com.ar/17ymh_venta_depart...,2 ambientes al frente con balcon cocina separa...,departamento en venta moron / moron (a197 185),NaN,2
32070,32070,sell,apartment,mar del plata,|argentina|buenos aires costa atlántica|mar de...,argentina,buenos aires costa atlántica,3430863.0,NaN,NaN,NaN,180000.0,usd,3176010.0,180000.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,http://www.properati.com.ar/185th_venta_depart...,"departamento 2 ambientes, buenos aires costa a...",departamento 2 amb con cochera. zona centro,https://thumbs4.properati.com/1/7icvsqmdwnzwjs...,2
57753,57753,sell,apartment,tigre,|argentina|bs.as. g.b.a. zona norte|tigre|tigre|,argentina,bs.as. g.b.a. zona norte,NaN,"-34.47571274,-58.61446381",-34.475713,-58.614464,135000.0,usd,2382007.5,135000.0,53.0,53.0,2547.169811,2547.169811,NaN,2.0,NaN,http://www.properati.com.ar/19u1l_venta_depart...,excelente complejo con vista al rio reconquist...,palmas plaza 100,https://thumbs4.properati.com/3/delvyh2ede1_-n...,2
51385,51385,sell,apartment,posadas,|argentina|misiones|posadas|,argentina,misiones,3429886.0,"-27.361931497,-55.9010519752",-27.361931,-55.901052,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,9.0,2.0,NaN,http://www.properati.com.ar/19grt_venta_depart...,venta de departamento 2 ambientes en posadas. ...,departamento en venta,https://thumbs4.properati.com/4/spzshqgbbddmlg...,2


In [193]:
list(aux_2_rooms_coincide.description)

['venta de departamento 1 dorm con cochera en victoria, san fernandocoordinar al celular  con alejandrolindisimo dto a estrenar en 2do piso contrafrente muy silencioso con vista a la arboleda. 2 ambientes grande, amplio living comedor, cocina independiente, dormitorio en suite, toilette, piso radiante con caldera individual, amplio balcón terraza con parrilla, cochera fija. -  - santiago jannot',
 '2 ambientes al frente con balcon cocina separada. deptos. de 1; 2 y 3 ambientes. 50 % al boleto saldo 18 cuotas fijas en pesos. posesion diciembre de 2018. consultar precio contado - publicado a traves de visualgestion - ',
 'departamento 2 ambientes, buenos aires costa atlántica, mar del plata, por u$s 180.000',
 'excelente complejo con vista al rio reconquista, a cien metros de av. liniers y av. libertador. cocheras cubiertas, jardín y pileta.departamento de 2 ambientes.',
 'venta de departamento 2 ambientes en posadas. financiado en 48 cuotas!departamentos de un dormitorio. muebles de coc

<h5>Se comprueba con varios samples que cuando ambos coinciden es porque el dato es correcto en ambas columnas

<a id="1.1.b"></a>
#### 1.1.b) Inmuebles de 3 ambientes

[Ir a índice](#indice)

In [194]:
# chequeo cantidades entre lo que el regex devuelve para tres ambientes y lo que dice el rooms
dos_ambientes = data[(data['ambientes_regex'] == 'tres') | (data['ambientes_regex'] == '3')]
print('Cantidades que me tira el regex')
print(dos_ambientes.ambientes_regex.value_counts())
print('')
print('Cantidades que me tira el rooms')
print(dos_ambientes.rooms.value_counts())
print('')
print('Cantidades que me tira el rooms en %')
print(dos_ambientes.rooms.value_counts(normalize=True))

Cantidades que me tira el regex
3       12150
tres     1359
Name: ambientes_regex, dtype: int64

Cantidades que me tira el rooms
0.0     6973
3.0     5775
2.0      336
4.0      165
1.0      137
5.0       61
6.0       33
7.0       13
8.0        7
10.0       5
9.0        2
15.0       1
32.0       1
Name: rooms, dtype: int64

Cantidades que me tira el rooms en %
0.0     0.516174
3.0     0.427493
2.0     0.024872
4.0     0.012214
1.0     0.010141
5.0     0.004516
6.0     0.002443
7.0     0.000962
8.0     0.000518
10.0    0.000370
9.0     0.000148
15.0    0.000074
32.0    0.000074
Name: rooms, dtype: float64


In [195]:
# chequeo sample de los inmuebles que el regex tiró tres ambientes y el rooms dice algo distinto
aux_3_rooms = data[((data['ambientes_regex'] == 'tres') | (data['ambientes_regex'] == '3')) & (data['rooms'] != 3)].sample(5)
aux_3_rooms

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
75717,75717,sell,apartment,belgrano,|argentina|capital federal|belgrano|,argentina,capital federal,3436077.0,"-34.5596536575,-58.4450904705",-34.559654,-58.445090,164500.0,usd,2902520.25,164500.0,70.0,66.0,2350.000000,2492.424242,NaN,0.0,2400.0,http://www.properati.com.ar/1aqmk_venta_depart...,departamento en venta de 3 ambientes en belgra...,libertador,https://thumbs4.properati.com/2/ry-d6s0fs8esxh...,3
18046,18046,sell,apartment,barrio norte,|argentina|capital federal|barrio norte|,argentina,capital federal,3436109.0,"-34.5970502,-58.3988816",-34.597050,-58.398882,230000.0,usd,4058235.00,230000.0,96.0,NaN,2395.833333,NaN,6.0,0.0,NaN,http://www.properati.com.ar/174ui_venta_depart...,venta de departamento de 3 ambientes con posib...,departamento en venta,https://thumbs4.properati.com/7/mvsvqypp5k5nwf...,3
102101,102101,sell,house,lanús,|argentina|bs.as. g.b.a. zona sur|lanús|lanús|,argentina,bs.as. g.b.a. zona sur,NaN,NaN,NaN,NaN,120000.0,usd,2117340.00,120000.0,100.0,100.0,1200.000000,1200.000000,NaN,0.0,NaN,http://www.properati.com.ar/1bq74_venta_casa_l...,"""""""""""""""""""""""""""""casa 3 ambientes c/cochera""""""""""""...",casa - lanús este,https://thumbs4.properati.com/0/jzjyg3l_zkj0qc...,3
110798,110798,sell,apartment,liniers,|argentina|capital federal|liniers|,argentina,capital federal,3431333.0,"-34.6390109,-58.5197498",-34.639011,-58.519750,160000.0,usd,2823120.00,160000.0,71.0,71.0,2253.521127,2253.521127,NaN,0.0,2800.0,http://www.properati.com.ar/1c4re_venta_depart...,codigo: 1066-luye3 ubicado en: martiniano legu...,3 amb. al contrafrente- suite- balcón impec a ...,https://thumbs4.properati.com/4/oa6cxynd9839nd...,3
28314,28314,sell,apartment,olivos,|argentina|bs.as. g.b.a. zona norte|vicente ló...,argentina,bs.as. g.b.a. zona norte,3430310.0,NaN,NaN,NaN,224726.0,usd,3965177.90,224726.0,76.0,69.0,2956.921053,3256.898551,3.0,0.0,NaN,http://www.properati.com.ar/17vzz_venta_depart...,"-- yrigoyen boulevard --amplio 3 amb de 69,50 ...","departamento 76m² con plantas en maipu, av. 18...",https://thumbs4.properati.com/4/8rlukfm_h02rr9...,3


In [196]:
list(aux_3_rooms.description)

['departamento en venta de 3 ambientes en belgrano. cabael departamento se encuentra en una inmejorable ubicación, cerca de varias lineas de colectivo y subte, cerca al centro comercial y accesos a autopistas.está impecable, listo para mudarse, es luminoso ya que tiene vista abierta y se distribuye de la siguiente manera: living comedor amplio, cocina separada con lavadero aparte y cubierto, dos habitaciones amplias con placeres y a.a, baño completo y balcón al contrarreste.tiene baulera...excelente valor! oportunidad!!!expensas: $2.400.- con aysa incluido!abl: $500.-ver plano:https://goo.gl/xdhvskver video:https://goo.gl/5g2oomver publicación:https://goo.gl/19rfghcifarelli | constructora - inmobiliaria m.n. 6199lunes a viernes de 10 a 13 y 15 a 19 | sábados 10 a 13código de la propiedad: cap308560las medidas son aproximadasqueres saber mas sobre nosotros? mira este video sobre nuestra filosofía de trabajo!https://goo.gl/d6kvn8',
 'venta de departamento de 3 ambientes con posibilidad d

Sigue siendo más fiable la columna "ambientes_regex".

In [197]:
# chequeo sample de los inmuebles que el regex tiró tres ambientes y el rooms dice tres ambientes
aux_3_rooms_coincide = data[((data['ambientes_regex'] == 'tres') | (data['ambientes_regex'] == '3')) & (data['rooms'] == 3)].sample(5)
aux_3_rooms_coincide

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
56107,56107,sell,apartment,palermo,|argentina|capital federal|palermo|,argentina,capital federal,3430234.0,"-34.5873916,-58.4125415",-34.587392,-58.412542,270000.0,usd,4764015.0,270000.0,82.0,78.0,3292.682927,3461.538462,NaN,3.0,7500.0,http://www.properati.com.ar/19q8f_venta_depart...,venta de departamento 3 ambientes con dependen...,departamento en venta,https://thumbs4.properati.com/7/ddldvqkpm_flxh...,3
24050,24050,sell,apartment,mar del plata,|argentina|buenos aires costa atlántica|mar de...,argentina,buenos aires costa atlántica,3430863.0,"-37.9821726,-57.5741446",-37.982173,-57.574145,67000.0,usd,1182181.5,67000.0,55.0,51.0,1218.181818,1313.725490,NaN,3.0,NaN,http://www.properati.com.ar/17nga_venta_depart...,depto 3 amb (51m2). piso 13º balcón terraza co...,depto 3 amb. vista panorámica,https://thumbs4.properati.com/3/iswfj2ewakqvl9...,3
41374,41374,sell,ph,caseros,|argentina|bs.as. g.b.a. zona oeste|tres de fe...,argentina,bs.as. g.b.a. zona oeste,3862356.0,"-34.601429,-58.557648",-34.601429,-58.557648,130000.0,usd,2293785.0,130000.0,NaN,63.0,NaN,2063.492063,NaN,3.0,NaN,http://www.properati.com.ar/18u36_venta_ph_cas...,corredor responsable: laura cepeda - csi 6203c...,ph 3 ambientes en casero con cochera descubierta,https://thumbs4.properati.com/4/cntxlusorlmcgu...,3
189,189,sell,apartment,villa bosch,|argentina|bs.as. g.b.a. zona oeste|tres de fe...,argentina,bs.as. g.b.a. zona oeste,3427463.0,"-34.5790672286,-58.5855497424",-34.579067,-58.585550,75000.0,usd,1323337.5,75000.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,http://www.properati.com.ar/15dfh_venta_depart...,departamento 3 ambientes en segundo piso al fr...,depto 3 ambientes en segundo piso al frente,https://thumbs4.properati.com/1/qfsjjztgr6xhy0...,3
9562,9562,sell,apartment,palermo,|argentina|capital federal|palermo|,argentina,capital federal,3430234.0,"-34.5855804,-58.4232627",-34.585580,-58.423263,430000.0,usd,7587135.0,430000.0,89.0,89.0,4831.460674,4831.460674,NaN,3.0,NaN,http://www.properati.com.ar/164q1_venta_depart...,espectacular depto de 3 amb -piso alto.en torr...,"venta torre quo paraguay 3 amb, mas cochera, a...",https://thumbs4.properati.com/0/0mvcbtrj9huiqr...,3


In [198]:
list(aux_3_rooms_coincide.description)

['venta de departamento 3 ambientes con dependencia y cochera en palermomuy lindo departamento ubicado sobre la avenida santa fe al contrafrente, luminoso y silencioso! posee un amplio living comedor con balcon. dos amplios dormitorios con placard mas un baño completo. la cocina es independiente, la dependencia se transformo en un practico comedor diario mas escritorio y el en baño de servicio se encuentra la conexión para lavaropas. el edificio posee servicios centrales, portería de 8 a 20hs y servicio de parking las 24hs. el departamento incluye un espacio guarda coche y una baulera. (las expensas incluyen cuota extraordinaria)  mas info al  ** tasamos hoy!***se deja constancia que las medidas (tanto totales como parciales), superficies, m2 y proporciones indicadas en el presente son aproximadas y sujetas a verificación, variación y/o ajuste. el precio de la presente operación y/o de las expensas aquí indicadas están sujetos a modificación sin previo aviso. -  - krell brokers',
 'dep

Se comprueba con varios samples que cuando ambos coinciden es porque el dato es correcto en ambas columnas

<a id="1.1.c"></a>
#### 1.1.c) Inmuebles de 1 ambiente

[Ir a índice](#indice)

In [199]:
# chequeo cantidades entre lo que el regex devuelve para un ambiente y lo que dice el rooms
dos_ambientes = data[(data['ambientes_regex'] == 'un') | (data['ambientes_regex'] == '1') | (data['ambientes_regex'] == 'mono')]
print('Cantidades que me tira el regex')
print(dos_ambientes.ambientes_regex.value_counts())
print('')
print('Cantidades que me tira el rooms')
print(dos_ambientes.rooms.value_counts())
print('')
print('Cantidades que me tira el rooms en %')
print(dos_ambientes.rooms.value_counts(normalize=True))

Cantidades que me tira el regex
mono    7404
1       2638
un      1013
Name: ambientes_regex, dtype: int64

Cantidades que me tira el rooms
0.0     5866
1.0     4392
2.0      490
3.0      118
4.0       89
5.0       43
6.0       20
8.0       13
7.0        7
10.0       6
11.0       3
12.0       2
9.0        2
18.0       1
14.0       1
25.0       1
13.0       1
Name: rooms, dtype: int64

Cantidades que me tira el rooms en %
0.0     0.530620
1.0     0.397286
2.0     0.044324
3.0     0.010674
4.0     0.008051
5.0     0.003890
6.0     0.001809
8.0     0.001176
7.0     0.000633
10.0    0.000543
11.0    0.000271
12.0    0.000181
9.0     0.000181
18.0    0.000090
14.0    0.000090
25.0    0.000090
13.0    0.000090
Name: rooms, dtype: float64


In [200]:
# chequeo sample de los inmuebles que el regex tiró un ambiente y el rooms dice algo distinto
aux_1_rooms = data[((data['ambientes_regex'] == 'un') | (data['ambientes_regex'] == '1') | (data['ambientes_regex'] == 'mono')) & (data['rooms'] != 1)].sample(5)
aux_1_rooms

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
108121,108121,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|la ...,argentina,bs.as. g.b.a. zona sur,3432039.0,NaN,NaN,NaN,85000.0,usd,1499782.50,85000.0,NaN,44.0,NaN,1931.818182,NaN,0.0,NaN,http://www.properati.com.ar/1bzhv_venta_depart...,departamento a estrenar . monoambiente contraf...,departamento - la plata,https://thumbs4.properati.com/8/td_cl9sqn9w--l...,mono
92539,92539,sell,apartment,belgrano,|argentina|capital federal|belgrano|,argentina,capital federal,3436077.0,"-34.5546094807,-58.4392982801",-34.554609,-58.439298,125000.0,usd,2205562.50,125000.0,42.0,38.0,2976.190476,3289.473684,NaN,0.0,1500.0,http://www.properati.com.ar/1bcz1_venta_depart...,departamento monoambiente en venta en belgrano...,sucre al 900 monoambiente divisible venta - be...,https://thumbs4.properati.com/1/sqcmni-g9xlmvo...,mono
105074,105074,sell,apartment,pinamar,|argentina|buenos aires costa atlántica|pinamar|,argentina,buenos aires costa atlántica,3429971.0,NaN,NaN,NaN,50000.0,usd,882225.00,50000.0,23.0,20.0,2173.913043,2500.000000,NaN,0.0,NaN,http://www.properati.com.ar/1bu4o_venta_depart...,venta departamento - calle jason entre marco ...,departamento - pinamar,https://thumbs4.properati.com/9/lnewoiehb6tqvt...,mono
33432,33432,sell,house,los polvorines,|argentina|bs.as. g.b.a. zona norte|malvinas a...,argentina,bs.as. g.b.a. zona norte,NaN,"-34.496172,-58.694436",-34.496172,-58.694436,115000.0,usd,2029117.50,115000.0,NaN,128.0,NaN,898.437500,NaN,4.0,NaN,http://www.properati.com.ar/189bp_venta_casa_l...,corredor responsable: uno bienes raices srl - ...,casa 4 amb. los polvorines excelente ubicación...,https://thumbs4.properati.com/6/_zbtnqtdvvk2cg...,1
60605,60605,sell,apartment,caballito,|argentina|capital federal|caballito|,argentina,capital federal,3435874.0,"-34.622973,-58.439913",-34.622973,-58.439913,124686.0,usd,2200022.12,124686.0,43.0,39.0,2899.674419,3197.076923,8.0,0.0,NaN,http://www.properati.com.ar/1a3v1_venta_depart...,venta departamento 1 ambiente caballitounidad ...,departamento en venta,https://thumbs4.properati.com/0/vvw4harcohxdvl...,1


In [201]:
list(aux_1_rooms.description)

['departamento a estrenar . monoambiente contrafrente',
 'departamento monoambiente en venta en belgrano chicomonoambiente al contrafrente, divisible y muy luminoso con balcón.el edificio cuenta con terraza de uso común con solarium.su ubicación es privilegiada ya que cuenta con excelentes accesos, a 500 mts avenida del libertador y a 300 mts avenida figueroa alcorta, lagos de palermo y polo gastronómico.mas info comunicarse al 011 tenes que vender para comprar?solicita hoy mismo tu tasación a  basa inmobiliaria .ar todas las medidas enunciadas son meramente orientativas, las medidas exactas serán las que se expresen en el respectivo título de propiedad de cada inmueble.',
 'venta departamento -  calle jason entre marco polo y simbad el marino - pinamar excelente monoambiente en pinamar, partido de la costa, de 23,23m2 totales desarrollados en ambiente principal, cocina separada, lavaderito integrado y balcón al frente.',
 'corredor responsable: uno bienes raices srl - cmcpsi 5821conta

<h5>Con los inmuebles de un ambiente suele ser más fiable el "departamento_regex" exceptuando cuando el tipo de inmueble es "casa". Debajo busco confirmarlo

Primero, ¿Cuántos son? 

In [202]:
aux_1_rooms_house = data[((data['ambientes_regex'] == 'un') | (data['ambientes_regex'] == '1') | (data['ambientes_regex'] == 'mono')) & (data['rooms'] != 1) & (data['property_type'] == 'house')]
aux_1_rooms_house.shape

(467, 27)

In [203]:
# ¿Y si no importa qué diga el "rooms", varía mucho el total?
data[((data['ambientes_regex'] == 'un') | (data['ambientes_regex'] == '1') | (data['ambientes_regex'] == 'mono')) & (data['property_type'] == 'house')].shape

(515, 27)

¿Sobre cuántas casas?

In [204]:
data[(data['property_type'] == 'house')].shape

(40268, 27)

In [205]:
aux_1_rooms_house.shape[0] / data[(data['property_type'] == 'house')].shape[0]

0.011597298102711831

In [206]:
# ¿Y sobre propiedades en total?
aux_1_rooms_house.shape[0] / data.shape[0]

0.0038524995875268107

In [207]:
# chequeo sample de los inmuebles que el regex tiró un ambiente y el rooms dice algo distinto para casos en los que la propiedad es una casa
aux_1_rooms_house_sample = aux_1_rooms_house.sample(5)
aux_1_rooms_house_sample

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
60748,60748,sell,house,confluencia,|argentina|neuquén|confluencia|,argentina,neuquén,3860360.0,"-38.9765,-68.062293",-38.976500,-68.062293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,http://www.properati.com.ar/1a46d_venta_casa_c...,corredor responsable: silvina gallegos - matri...,jardines del rey,https://thumbs4.properati.com/9/rot1mxlcgl8xlm...,un
8095,8095,sell,house,funes,|argentina|santa fe|funes|,argentina,santa fe,3855302.0,"-32.9243089067,-60.8273347075",-32.924309,-60.827335,NaN,NaN,NaN,NaN,800.0,320.0,NaN,NaN,NaN,0.0,NaN,http://www.properati.com.ar/15zw2_venta_casa_f...,descripcióningreso : entrada vehicular imperia...,funes hills - san marino,https://thumbs4.properati.com/5/a012pobavr1r47...,un
46515,46515,sell,house,benavidez,|argentina|bs.as. g.b.a. zona norte|tigre|bena...,argentina,bs.as. g.b.a. zona norte,3436048.0,NaN,NaN,NaN,720000.0,usd,12704040.00,720000.00,1068.0,300.0,674.157303,2400.000000,NaN,0.0,NaN,http://www.properati.com.ar/1967s_venta_casa_b...,"hermosa casa en el barrio san marco,construida...","casa 1068m² con cochera, tigre, benavidez, por...",https://thumbs4.properati.com/0/fm6eqvrzat0qg2...,un
11,11,sell,house,san miguel,|argentina|bs.as. g.b.a. zona norte|san miguel|,argentina,bs.as. g.b.a. zona norte,NaN,"-34.5390571,-58.7196093",-34.539057,-58.719609,150000.0,usd,2646675.00,150000.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,http://www.properati.com.ar/15bp0_venta_casa_s...,"entrada de coche, jardin, living en desnivel, ...",chalet,https://thumbs4.properati.com/4/ph0mns_2zcgohr...,1
16182,16182,sell,house,moreno,|argentina|bs.as. g.b.a. zona oeste|moreno|,argentina,bs.as. g.b.a. zona oeste,3430550.0,"-34.6339063,-58.7881789",-34.633906,-58.788179,699900.0,ars,692598.85,39252.96,240.0,70.0,163.554000,9998.571429,NaN,0.0,NaN,http://www.properati.com.ar/16xs4_venta_casa_m...,codigo: 3484-ref. 3348(2) ubicado en: panama 5...,casa en moreno a 50 mts de colectora oportunid...,https://thumbs4.properati.com/9/9ndmnf1vhbfi_2...,1


In [208]:
list(aux_1_rooms_house_sample.description)

['corredor responsable: silvina gallegos - matricula n° 392contacto: gabriela mander - mls id # 420371081-1equipo re/max sinergia vende propiedad en barrio jardines del rey, uno de los barrios mas cotizados de la ciudad de neuquén.  superficie terreno :  800 m2 superficie construida : 240 m2 cubiertos y 30 m2 semicubiertos la casa se desarrolla en  dos plantas:  planta baja :  - living individual - cocina comedor separados por barra desayunadora con vista a la galería y jardín. - lavadero completamente amueblado. - dormitorios :  tres, subiendo una pequeña escalera para llegar a un ambiente mas íntimo. - todas las habitaciones tienen con vista al jardín.  - baños : dos , uno con antebaño y el otro principal en suite. - quincho con parrilla grande y habitáculo para asador. - cochera : para dos autos con sistema automatizado. planta alta: habitación muy agradable tipo altillo, con biblioteca apta para escritorio o sala de juegos.  gran parque con sistema de riego automatizado.',
 'descri

In [209]:
# chequeo sample de los inmuebles que el regex tiró un ambiente y el rooms dice algo distinto para casos en los que la propiedad es un departamento
aux_1_rooms_nothouse = data[((data['ambientes_regex'] == 'un') | (data['ambientes_regex'] == '1') | (data['ambientes_regex'] == 'mono')) & (data['rooms'] != 1) & (data['property_type'] == 'apartment')].sample(5)
aux_1_rooms_nothouse

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
79977,79977,sell,apartment,rosario,|argentina|santa fe|rosario|,argentina,santa fe,3838574.0,"-32.9370358,-60.677219",-32.937036,-60.677219,NaN,NaN,NaN,NaN,60.0,60.0,NaN,NaN,NaN,0.0,NaN,http://www.properati.com.ar/1aw9i_venta_depart...,sergio villella inmobiliaria presenta departam...,lavalle 600 - departamento dos dormitorios en ...,https://thumbs4.properati.com/6/hq3wkevcwnoc4-...,mono
7723,7723,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|la ...,argentina,bs.as. g.b.a. zona sur,3432039.0,"-34.9209282251,-57.9599405802",-34.920928,-57.959941,68000.0,usd,1199826.00,68000.0,43.0,39.0,1581.395349,1743.589744,NaN,0.0,700.0,http://www.properati.com.ar/15ywi_venta_depart...,venta de departamento monoambiente en la plata...,departamento en venta,https://thumbs4.properati.com/2/q2np7mwv9gccxf...,mono
105814,105814,sell,apartment,rosario,|argentina|santa fe|rosario|,argentina,santa fe,3838574.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,0.0,450.0,http://www.properati.com.ar/1bv9y_venta_depart...,"monoambiente desarrollado en un cuarto piso, a...",departamento - centro,https://thumbs4.properati.com/9/kl8isrdxlhtbph...,mono
61868,61868,sell,apartment,pilar,|argentina|bs.as. g.b.a. zona norte|pilar|,argentina,bs.as. g.b.a. zona norte,3429979.0,NaN,NaN,NaN,150000.0,usd,2646675.00,150000.0,58.0,52.0,2586.206897,2884.615385,NaN,2.0,NaN,http://www.properati.com.ar/1a6a3_venta_depart...,lagoon pilar una vida all inclusive!un lugar p...,departamento en venta en lagoon pilar 2 amb co...,https://thumbs4.properati.com/6/ea9k-p6alc6q_b...,un
80498,80498,sell,apartment,san telmo,|argentina|capital federal|san telmo|,argentina,capital federal,3428113.0,NaN,NaN,NaN,84240.0,usd,1486372.68,84240.0,36.0,33.0,2340.000000,2552.727273,NaN,0.0,NaN,http://www.properati.com.ar/1awwt_venta_depart...,san telmo - departamento en venta de 1 ambient...,san telmo - departamento en venta de 1 ambient...,https://thumbs4.properati.com/8/zgkywdesba9bcm...,1


In [210]:
list(aux_1_rooms_nothouse.description)

['sergio villella inmobiliaria presenta departamentos en venta en el edificio lavalle, ubicado sobre calle lavalle al 600, entre las calles san lorenzo y santa fe, a una cuadra de bv. avellaneda y a pocas de la terminal de ómnibus mariano moreno. cercano a la facultad de medicina de la unr, cuenta con una ubicación estratégica ya que está emplazado en una zona que se ubica a minutos de los distintos puntos cardinales de la ciudad de rosario.el edificio ofrece unidades en venta monoambiente, de 1 y 2 dormitorios, todas con balcón. en la planta baja del edificio se encuentran las cocheras que son opcionales y un local comercial.forma de pago: anticipo del 50% saldo hasta 24 cuota en pesos + icac.posesión inmediata.departamento 2 dormitorios frente con balcón.',
 'venta de departamento monoambiente en la plata, la plata.venta de departamento. amplio monoambiente, baño completo, cocina integrada separada con barra desayunadora, balcón. pisos de porcelanato en el baño y la cocina, en el est

<h5>Se confirma que cuando se trata de property_type != house, sigue siendo más fiable la columna "ambientes_regex".

In [211]:
# chequeo sample de los inmuebles que el regex tiró un ambiente y el rooms dice un ambiente
aux_1_rooms_coincide = data[((data['ambientes_regex'] == 'un') | (data['ambientes_regex'] == '1') | (data['ambientes_regex'] == 'mono')) & (data['rooms'] == 1)].sample(5)
aux_1_rooms_coincide

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
49710,49710,sell,apartment,belgrano,|argentina|capital federal|belgrano|,argentina,capital federal,3436077.0,"-34.5534737,-58.4540257",-34.553474,-58.454026,111400.0,usd,1965597.30,111400.00,37.0,30.0,3010.810811,3713.333333,4.0,1.0,2600.0,http://www.properati.com.ar/19cov_venta_depart...,venta de departamento 1 ambiente monoambiente ...,departamento en venta,https://thumbs4.properati.com/5/obi1vyg644k8fn...,1
52656,52656,sell,apartment,mar del plata,|argentina|buenos aires costa atlántica|mar de...,argentina,buenos aires costa atlántica,3430863.0,"-38.0615893,-57.5474308",-38.061589,-57.547431,55000.0,usd,970447.50,55000.00,33.0,33.0,1666.666667,1666.666667,NaN,1.0,NaN,http://www.properati.com.ar/19jn2_venta_depart...,departamento de 1 ambiente con vista al mar. a...,depto. 1 amb. vista al mar - nuevo -punta mogotes,https://thumbs4.properati.com/3/fbx6ngwxxvxb3r...,1
62177,62177,sell,apartment,flores,|argentina|capital federal|flores|,argentina,capital federal,3433918.0,"-34.6375502,-58.4575273",-34.637550,-58.457527,118000.0,usd,2082051.00,118000.00,53.0,46.0,2226.415094,2565.217391,NaN,1.0,NaN,http://www.properati.com.ar/1a6ry_venta_depart...,edificio ubicado en zona residencial de flores...,departamento venta,https://thumbs4.properati.com/5/vf9-yw4zdb64uy...,mono
26389,26389,sell,apartment,boedo,|argentina|capital federal|boedo|,argentina,capital federal,3436003.0,NaN,NaN,NaN,632578.0,usd,11161522.52,632578.00,30.0,30.0,21085.933333,21085.933333,NaN,1.0,NaN,http://www.properati.com.ar/17r1z_venta_depart...,lan869- un lugar con todos los servicios en cu...,lan869- un lugar con todos los servicios en cu...,https://thumbs4.properati.com/6/ebqvveav9p3tdg...,1
113456,113456,sell,apartment,palermo,|argentina|capital federal|palermo|,argentina,capital federal,3430234.0,"-34.5807733,-58.4070408",-34.580773,-58.407041,2068878.0,ars,2047296.27,116030.28,40.0,36.0,2900.757000,57468.833333,NaN,1.0,NaN,http://www.properati.com.ar/1c8dq_venta_depart...,departamento de 1 ambiente al contrafrente.con...,"gran oportunidad en pozo, monoambiente ameniti...",https://thumbs4.properati.com/7/qvwh92lska3z0y...,1


In [212]:
list(aux_1_rooms_coincide.description)

['venta de departamento 1 ambiente monoambiente  en belgrano, capital federalvivienda o estudio profesional - a estrenar entrega inmediata.al frente con amplio balcon vidriado, mucho sol, baño completo, cocina integrada, ambiente divisible.excelente calidad constructiva, agua caliente central, calefacción por losa radiante.a dos cuadras de av. del libertador, proximo zona gastronomica.amenities en ultimo piso; piscina con solarium y duchas, sauna, ducha escocesa, sum y parrilla.programe una visita con nosotros.0810-220-1897 - sabados, domingos y feriados 15- / 15-ricardo maidana empresa inmobiliariaventas y tasaciones 0810-220- -  - ricardo maidana empresa inmobiliaria',
 'departamento de 1 ambiente con vista al mar. ambiente cómodo, cocina con barra desayunadora, baño completo. edificio nuevo. electrodependiente. se vende amoblado . ideal inversión. ',
 'edificio ubicado en zona residencial de flores, compuesto por doce departamentos de categoría a estrenar: cuatro monoambientes divis

<h5>Se comprueba con varios samples que cuando ambos coinciden es porque el dato es correcto en ambas columnas

<a id="1.1.d"></a>
#### 1.1.d) Inmuebles de 4 ambientes

[Ir a índice](#indice)

In [213]:
# chequeo cantidades entre lo que el regex devuelve para cuatro ambientes y lo que dice el rooms
dos_ambientes = data[(data['ambientes_regex'] == 'cuatro') | (data['ambientes_regex'] == '4')]
print('Cantidades que me tira el regex')
print(dos_ambientes.ambientes_regex.value_counts())
print('')
print('Cantidades que me tira el rooms')
print(dos_ambientes.rooms.value_counts())
print('')
print('Cantidades que me tira el rooms en %')
print(dos_ambientes.rooms.value_counts(normalize=True))

Cantidades que me tira el regex
4         6728
cuatro     304
Name: ambientes_regex, dtype: int64

Cantidades que me tira el rooms
0.0     3815
4.0     2736
3.0      234
2.0      101
5.0       84
1.0       48
7.0        6
6.0        5
12.0       1
8.0        1
9.0        1
Name: rooms, dtype: int64

Cantidades que me tira el rooms en %
0.0     0.542520
4.0     0.389078
3.0     0.033276
2.0     0.014363
5.0     0.011945
1.0     0.006826
7.0     0.000853
6.0     0.000711
12.0    0.000142
8.0     0.000142
9.0     0.000142
Name: rooms, dtype: float64


In [214]:
# chequeo sample de los inmuebles que el regex tiró cuatro ambientes y el rooms dice algo distinto
aux_4_rooms = data[((data['ambientes_regex'] == 'cuatro') | (data['ambientes_regex'] == '4')) & (data['rooms'] != 4)].sample(5)
aux_4_rooms

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
35774,35774,sell,apartment,san nicolás,|argentina|capital federal|san nicolás|,argentina,capital federal,6693229.0,"-34.601284,-58.378012",-34.601284,-58.378012,108000.0,usd,1905606.00,108000.0,60.0,60.0,1800.000000,1800.000000,NaN,2.0,NaN,http://www.properati.com.ar/18f7u_venta_depart...,señorial semipiso de 4 ambientes en plena zona...,departamento venta,https://thumbs4.properati.com/6/xvzx98qzc6ymym...,4
31164,31164,sell,house,valentín alsina,|argentina|bs.as. g.b.a. zona sur|lanús|valent...,argentina,bs.as. g.b.a. zona sur,NaN,"-34.67244,-58.4086297",-34.672440,-58.408630,250000.0,usd,4411125.00,250000.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,http://www.properati.com.ar/183jk_venta_casa_v...,"venta de casa 4 ambientes en valentín alsina, ...",casa en venta,https://thumbs4.properati.com/6/vmozi0nxxlhdpt...,4
74396,74396,sell,apartment,flores,|argentina|capital federal|flores|,argentina,capital federal,3433918.0,NaN,NaN,NaN,500000.0,usd,8822250.00,500000.0,160.0,295.0,3125.000000,1694.915254,NaN,0.0,NaN,http://www.properati.com.ar/1aox3_venta_depart...,venta departamento 4 ambientes en el barrio de...,departamento en flores,https://thumbs4.properati.com/6/5x2k4me6l-za-z...,4
92588,92588,sell,apartment,belgrano,|argentina|capital federal|belgrano|,argentina,capital federal,3436077.0,"-34.5679209469,-58.4428058518",-34.567921,-58.442806,699000.0,usd,12333505.50,699000.0,202.0,180.0,3460.396040,3883.333333,NaN,0.0,8500.0,http://www.properati.com.ar/1bd17_venta_depart...,"departamento de 4 ambientes con dependencia, b...",departamento duplex 4 ambientes con 2 cocheras...,https://thumbs4.properati.com/1/qbb8spmfuqg-va...,4
87599,87599,sell,apartment,parque patricios,|argentina|capital federal|parque patricios|,argentina,capital federal,3430114.0,NaN,NaN,NaN,105601.0,usd,1863276.84,105601.0,53.0,45.0,1992.471698,2346.688889,NaN,0.0,NaN,http://www.properati.com.ar/1b65a_venta_depart...,espai patricios se encuentra emplazado en el b...,departamento - 2 ambientes - parque patricios,https://thumbs4.properati.com/0/3sjyqznentjclc...,4


In [215]:
list(aux_4_rooms.description)

['señorial semipiso de 4 ambientes en plena zona negocios, turismo y residencial del centro de la ciudad. a pocas cuadras de catalinas, retiro, av. 9 de julio, teatro colón o tribunales.edificio clásico de arquitectura estilo francés, acceso por puerta de hierro y lobby en pb en mármol y espejos. sólo 2 unidades por piso, escalera de mármol y puerta de acceso a unidades en madera maciza.el semipiso cuenta con recepción o espera, dos despachos o ambientes muy amplios con ventanales al contrafrente, secretaría o escritorio y habitación adicional para archivos, espacio para office o lavadero y baño de época de generosas medidas.todo con excelente iluminación y ventilación. techos altos, carpintería interior en madera con puertas tablero y muros interiores con espesor adecuado para aislación acústica y térmica.muchas posibilidades de uso y funciones, ideal por precio para adaptar y decorar a criterio del propietario. excelente inversión para renta de oficina o turística y para opción de vi

Con los inmuebles de 4 ambientes suele ser más fiable el "departamento_regex"

In [216]:
# chequeo sample de los inmuebles que el regex tiró cuatro ambientes y el rooms dice cuatro ambientes
aux_4_rooms_coincide = data[((data['ambientes_regex'] == 'cuatro') | (data['ambientes_regex'] == '4')) & (data['rooms'] == 4)].sample(5)
aux_4_rooms_coincide

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
4559,4559,sell,apartment,wilde,|argentina|bs.as. g.b.a. zona sur|avellaneda|w...,argentina,bs.as. g.b.a. zona sur,3427326.0,"-34.6938125,-58.313027",-34.693812,-58.313027,95000.0,usd,1676227.5,95000.0,80.0,80.0,1187.500000,1187.500000,NaN,4.0,NaN,http://www.properati.com.ar/15n6b_venta_depart...,"departamento cuatro ambientes: 3 dormitorios, ...",piran 700 - wilde - avellaneda,https://thumbs4.properati.com/6/qzxx1hhnfmwu_y...,cuatro
17989,17989,sell,house,santos lugares,|argentina|bs.as. g.b.a. zona oeste|tres de fe...,argentina,bs.as. g.b.a. zona oeste,3428075.0,"-34.6030218,-58.5461051",-34.603022,-58.546105,295000.0,usd,5205127.5,295000.0,468.0,314.0,630.341880,939.490446,NaN,4.0,NaN,http://www.properati.com.ar/174pm_venta_casa_s...,venta de chalet de 4 ambientes con garage en s...,casa en venta,https://thumbs4.properati.com/7/pingmja52xx9lx...,4
418,418,sell,apartment,nordelta,|argentina|bs.as. g.b.a. zona norte|tigre|nord...,argentina,bs.as. g.b.a. zona norte,NaN,"-34.3967647,-58.6476628",-34.396765,-58.647663,300000.0,usd,5293350.0,300000.0,130.0,115.0,2307.692308,2608.695652,NaN,4.0,NaN,http://www.properati.com.ar/15ei2_venta_depart...,increíble opción de 4 ambientes en el complejo...,excelente quartier 4 amb,https://thumbs4.properati.com/7/pbciyyjmkrvl08...,4
9063,9063,sell,apartment,belgrano,|argentina|capital federal|belgrano|,argentina,capital federal,3436077.0,"-34.5566079,-58.4570689",-34.556608,-58.457069,360000.0,usd,6352020.0,360000.0,164.0,137.0,2195.121951,2627.737226,NaN,4.0,NaN,http://www.properati.com.ar/162tz_venta_depart...,arcos 2500 belgrano. piso de 4amb c/ cochera u...,arcos 2500 belgrano. piso de 4amb c/ cochera u...,https://thumbs4.properati.com/0/oqyh1hf-umrxe3...,4
51095,51095,sell,apartment,san nicolás,|argentina|capital federal|san nicolás|,argentina,capital federal,6693229.0,"-34.6056295,-58.3921787",-34.605629,-58.392179,179000.0,usd,3158365.5,179000.0,90.0,90.0,1988.888889,1988.888889,NaN,4.0,NaN,http://www.properati.com.ar/19g7n_venta_depart...,apto profesional. apto crédito. semipiso lumin...,sarmiento 1800 san nicolás 4amb u$179.000 apto...,https://thumbs4.properati.com/2/vucwk00xulz6zz...,4


In [217]:
list(aux_4_rooms_coincide.description)

['departamento cuatro ambientes: 3 dormitorios, cocina, living comedor, baño completo, lavadero, espacio guarda coches. apto crédito hipotecario.',
 'venta de chalet de 4 ambientes con garage en santos lugares. distribuido en dos plantas. en planta baja: amplio living comedor, cocina con comedor diario, toilette, dormitorio principal en suite con vestidor, jardin al fondo, quincho con parrilla y cocina equipada, habitacion de servicio con lavadero, cochera en galeria para 2 ó 3 autos. planta alta; escritorio o sala de estar balconeando a living, dormitorio en suite con baño completo y salida a patio terraza al contrafrente. lote: 8.66 x 42.00 aprox.- todos los ambientes separados, super luminosa. ** se deja constancia que las medidas y fotos consignadas son de caracter no contractual. francisco leguizamon inmuebles - tel:  / whatsapp: 11- // e-mail:  // oficina: dorrego 1190, lomas del mirador.- -  - francisco  leguizamon inmuebles',
 'increíble opción de 4 ambientes en el complejo qua

Se comprueba con varios samples que cuando ambos coinciden es porque el dato es correcto en ambas columnas

<a id="1.1.e"></a> 
#### 1.1.e) Dropear dataset según cantidad de ambientes

[Ir a índice](#indice)

<h5>Dado que tanto en la columna "rooms" como en la "ambientes_regex", los inmuebles con valor igual o mayor a 7 no alcanzan al 1% del dataset, voy a dropearlos según la columna generalmente más fiable: "ambientes_regex".

In [218]:
data.ambientes_regex.value_counts()

No se encontró    71335
2                 12507
3                 12150
mono               7404
4                  6728
dos                2808
1                  2638
5                  1889
tres               1359
un                 1013
6                   557
cuatro              304
7                   228
cinco                57
8                    38
9                    21
00                   18
02                   16
10                   14
11                   13
72                    8
12                    6
50                    6
13                    5
23                    5
14                    5
53                    5
04                    5
33                    4
24                    4
73                    4
42                    4
93                    3
83                    3
62                    3
28                    3
26                    3
18                    3
15                    3
22                    3
06                    3
36              

In [219]:
# primero limpio los datos existentes. EL "No se encontró" lo transformo a 0, sabiendo que significa que no está el dato, después decido qué hacer con eso
data["ambientes_regex"] = data.ambientes_regex.replace({'un': "1", 'mono': "1", 'tres': "3", 'dos': "2", 'cuatro': "4", 'cinco': "5", 'No se encontró': "0"})
data.ambientes_regex.value_counts()

0     71336
2     15315
3     13509
1     11055
4      7032
5      1946
6       557
7       228
8        38
9        21
00       18
02       16
10       14
11       13
72        8
12        6
50        6
13        5
23        5
14        5
53        5
04        5
73        4
24        4
33        4
42        4
06        3
15        3
83        3
62        3
28        3
26        3
18        3
22        3
93        3
25        2
17        2
90        2
36        2
21        2
49        2
47        1
76        1
84        1
51        1
03        1
97        1
52        1
92        1
81        1
61        1
44        1
65        1
20        1
16        1
71        1
54        1
64        1
74        1
27        1
32        1
46        1
85        1
Name: ambientes_regex, dtype: int64

In [220]:
data['ambientes_regex'] = data.ambientes_regex.astype('int64')
data.ambientes_regex.value_counts()

0     71354
2     15331
3     13510
1     11055
4      7037
5      1946
6       560
7       228
8        38
9        21
10       14
11       13
72        8
50        6
12        6
53        5
13        5
14        5
23        5
33        4
42        4
73        4
24        4
83        3
15        3
18        3
62        3
26        3
93        3
28        3
22        3
25        2
49        2
90        2
36        2
21        2
17        2
52        1
51        1
65        1
54        1
47        1
46        1
64        1
44        1
71        1
92        1
97        1
32        1
84        1
74        1
27        1
76        1
16        1
81        1
85        1
61        1
20        1
Name: ambientes_regex, dtype: int64

In [221]:
cantidad_anterior = data.shape[0]

In [222]:
data = data.loc[data['ambientes_regex'] < 7,:]
data.ambientes_regex.value_counts()

0    71354
2    15331
3    13510
1    11055
4     7037
5     1946
6      560
Name: ambientes_regex, dtype: int64

In [223]:
(cantidad_anterior - data.shape[0]) / cantidad_anterior

0.0035225210361326514

In [224]:
cantidad_anterior - data.shape[0]

427

<a id="1.2"></a>
#### 1.2) Limpieza para cuando el regex devuelve un ambiente pero es una casa

[Ir a índice](#indice)

Según el análisis hecho, también llegué a la conclusión de que cuando la propiedad es 'house' y el regex devuelve que es un monoambiente, es más fiable el dato de la columna "rooms". Sin embargo, leyendo varios samples y viendo que suelen ser casuísticas raras, más el hecho de que representan 1,1% del total de las casas y menos del 1% del total de las propiedades, decido dropearlas.

In [225]:
cantidad_anterior = data.shape[0]
cantidad_anterior

120793

In [226]:
# genero máscara 
mask_data = ((data['ambientes_regex'] == 1) & (data['property_type'] == 'house'))
mask_data.value_counts()

False    120278
True        515
dtype: int64

In [227]:
# necesito que los valores verdaderos sean falsos y viceversa, para sacar a los que indica la máscara en vez de quedármelos
mask_data = mask_data.replace({True: False, False: True})
mask_data.value_counts()

True     120278
False       515
dtype: int64

In [228]:
data = data[mask_data]
data.shape

(120278, 27)

In [229]:
cantidad_anterior - data.shape[0]

515

In [230]:
(cantidad_anterior - data.shape[0]) / cantidad_anterior

0.004263492089773414

<a id="1.3"></a>
#### 1.3) Imputación de valores en "ambientes_regex" y borrar "rooms"

[Ir a índice](#indice)

In [231]:
# dado que nos vamos a quedar con "ambientes_regex", borramos "roooms"
data = data.drop(columns = ['rooms'])
data.head(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex
0,0,sell,ph,mataderos,|argentina|capital federal|mataderos|,argentina,capital federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,usd,1093959.0,62000.0,55.0,40.0,1127.272727,1550.0,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 ambientes tipo casa planta baja por pasillo,...",2 amb tipo casa sin expensas en pb,https://thumbs4.properati.com/8/bluuyihjlhgiik...,2
1,1,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|,argentina,bs.as. g.b.a. zona sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,usd,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,venta de departamento en décimo piso al frente...,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpvbu2ztha7jv...,0


In [232]:
data.pivot_table(index=["ambientes_regex"],columns=['property_type'], values=['description'], aggfunc=len)
# ver cómo agregar una columna que totalice
# 

description                         
property_type     apartment    house      ph   store
ambientes_regex                                     
0                   31898.0  32680.0  2938.0  3838.0
1                   10294.0      NaN   171.0    75.0
2                   13842.0    769.0   650.0    70.0
3                   10036.0   2157.0  1238.0    79.0
4                    4235.0   2208.0   548.0    46.0
5                     541.0   1252.0   144.0     9.0
6                      80.0    437.0    38.0     5.0

<a id="2"></a>
## 2) Construcción de otras variables que podrían influir en el valor de inmueble

[Ir a índice](#indice)

In [233]:
# por ahora es un string lo que va a ser regex
patron_parrilla = "(?P<parrilla>.?parrilla|asador)"
patron_cochera = '(?P<cochera>.?cochera)'
patron_pileta = '(?P<pileta>.?pileta|piscina)'
patron_sum = '(?P<sum>.?sum)'
patron_solarium = '(?P<sum>.?solárium|solarium)'
patron_gym = '(?P<gym>.?gimnasio|gym)'
patron_balcon = '(?P<balcon>.?balcon|balcón)'
patron_jardin = '(?P<jardin>.?jardín|jardin)'
patron_lavadero = '(?P<lavadero>.?laundry|loundry|lavadero)'
patron_aestrenar = '(?P<estrenar>.?estrenar)'
patron_quincho = '(?P<quincho>.?quincho)'
patron_patio = '(?P<patio>.?patio)'
patron_en_pozo = '(?P<en_pozo>.?en\s?pozo)'

In [234]:
# el compile lo que hace es pasarlo a formato pattern, objeto que detecta el patrón que le pido
regex_parrilla  = re.compile(patron_parrilla)
regex_cochera  = re.compile(patron_cochera)
regex_pileta  = re.compile(patron_pileta)
regex_sum  = re.compile(patron_sum)
regex_gym  = re.compile(patron_gym)
regex_balcon  = re.compile(patron_balcon)
regex_jardin  = re.compile(patron_jardin)
regex_lavadero  = re.compile(patron_lavadero)
regex_aestrenar  = re.compile(patron_aestrenar)
regex_quincho  = re.compile(patron_quincho)
regex_patio  = re.compile(patron_patio)
regex_en_pozo  = re.compile(patron_en_pozo)

In [235]:
# defino una función para generar columnas con las distintas características de inmuebles y contabilizarlas

def columna_regex(regex, nombre_de_nueva_columna):
    columna_regex = data.description.apply(lambda x: None if x is np.NaN else regex.search(x))
    data[nombre_de_nueva_columna] = columna_regex.apply(lambda x: 0 if x is None else 1)
    print('Esta es la cantidad neta: ')
    print(data[nombre_de_nueva_columna].value_counts())
    print('')
    print('Esta es la cantidad en %: ')
    print(data[nombre_de_nueva_columna].value_counts(normalize=True))
    return

#### 2.1) Parrilla

In [236]:
columna_regex(regex_parrilla, 'parrilla_regex')

Esta es la cantidad neta: 
0    82460
1    37818
Name: parrilla_regex, dtype: int64

Esta es la cantidad en %: 
0    0.685578
1    0.314422
Name: parrilla_regex, dtype: float64


#### 2.2) Cochera

In [237]:
columna_regex(regex_cochera, 'cochera_regex')

Esta es la cantidad neta: 
0    78145
1    42133
Name: cochera_regex, dtype: int64

Esta es la cantidad en %: 
0    0.649703
1    0.350297
Name: cochera_regex, dtype: float64


#### 2.3) Pileta

In [238]:
columna_regex(regex_pileta, 'pileta_regex')

Esta es la cantidad neta: 
0    87818
1    32460
Name: pileta_regex, dtype: int64

Esta es la cantidad en %: 
0    0.730125
1    0.269875
Name: pileta_regex, dtype: float64


#### 2.4) SUM

In [239]:
columna_regex(regex_sum, 'sum_regex')

Esta es la cantidad neta: 
0    98512
1    21766
Name: sum_regex, dtype: int64

Esta es la cantidad en %: 
0    0.819036
1    0.180964
Name: sum_regex, dtype: float64


#### 2.5) Solarium

In [ ]:
# columna_regex(regex_sum, 'sum_regex')

#### 2.5) Gym

In [240]:
columna_regex(regex_gym, 'gym_regex')

Esta es la cantidad neta: 
0    111008
1      9270
Name: gym_regex, dtype: int64

Esta es la cantidad en %: 
0    0.922929
1    0.077071
Name: gym_regex, dtype: float64


Lo desestimaría dado que es demasiado bajo el porcentaje?

In [241]:
# data = data.drop(columns = ['gym_regex'])

#### 2.6) Balcón

In [242]:
columna_regex(regex_balcon, 'balcon_regex')

Esta es la cantidad neta: 
0    77360
1    42918
Name: balcon_regex, dtype: int64

Esta es la cantidad en %: 
0    0.643177
1    0.356823
Name: balcon_regex, dtype: float64


#### 2.7) Jardín

In [243]:
columna_regex(regex_jardin, 'jardin_regex')

Esta es la cantidad neta: 
0    103148
1     17130
Name: jardin_regex, dtype: int64

Esta es la cantidad en %: 
0    0.85758
1    0.14242
Name: jardin_regex, dtype: float64


#### 2.8) Lavadero

In [244]:
columna_regex(regex_lavadero, 'lavadero_regex')

Esta es la cantidad neta: 
0    75945
1    44333
Name: lavadero_regex, dtype: int64

Esta es la cantidad en %: 
0    0.631412
1    0.368588
Name: lavadero_regex, dtype: float64


#### 2.9) A estrenar

In [245]:
columna_regex(regex_aestrenar, 'estrenar_regex')

Esta es la cantidad neta: 
0    109709
1     10569
Name: estrenar_regex, dtype: int64

Esta es la cantidad en %: 
0    0.912129
1    0.087871
Name: estrenar_regex, dtype: float64


No sé si desestimarlo

#### 2.10) Quincho

In [246]:
columna_regex(regex_quincho, 'quincho_regex')

Esta es la cantidad neta: 
0    107414
1     12864
Name: quincho_regex, dtype: int64

Esta es la cantidad en %: 
0    0.893048
1    0.106952
Name: quincho_regex, dtype: float64


#### 2.11) Patio

In [247]:
columna_regex(regex_patio, 'patio_regex')

Esta es la cantidad neta: 
0    99112
1    21166
Name: patio_regex, dtype: int64

Esta es la cantidad en %: 
0    0.824024
1    0.175976
Name: patio_regex, dtype: float64


#### 2.12) En pozo + dropear

In [248]:
columna_regex(regex_en_pozo, 'en_pozo_regex')

Esta es la cantidad neta: 
0    118495
1      1783
Name: en_pozo_regex, dtype: int64

Esta es la cantidad en %: 
0    0.985176
1    0.014824
Name: en_pozo_regex, dtype: float64


In [249]:
# chequeo sample de los inmuebles que el regex tiró que son en pozo
en_pozo = data[(data['en_pozo_regex'] == 1)].sample(5)
en_pozo

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex,en_pozo_regex
61276,61276,sell,apartment,centro / microcentro,|argentina|capital federal|centro / microcentro|,argentina,capital federal,3435548.0,"-34.5988137211,-58.3757373568",-34.598814,-58.375737,93977.0,usd,1658177.17,93977.00,29.0,29.0,3240.586207,3240.586207,10.0,NaN,http://www.properati.com.ar/1a55w_venta_depart...,inversión emprendimiento en pozo. apart hotel ...,departamento en venta,https://thumbs4.properati.com/4/0ebd8j82nq7dfn...,1,1,1,0,1,0,0,0,0,0,0,0,1
77556,77556,sell,apartment,villa luro,|argentina|capital federal|villa luro|,argentina,capital federal,3427407.0,"-34.6395938,-58.5006643",-34.639594,-58.500664,179000.0,usd,3158365.50,179000.00,78.0,70.0,2294.871795,2557.142857,NaN,NaN,http://www.properati.com.ar/1aszh_venta_depart...,"venta departamento 3 ambientes en villa luro, ...",departamento - villa luro,https://thumbs4.properati.com/4/wlnktup-aip55q...,3,1,1,1,1,0,0,0,0,0,0,0,1
116201,116201,sell,apartment,confluencia,|argentina|neuquén|confluencia|,argentina,neuquén,3860360.0,"-38.949764,-68.051655",-38.949764,-68.051655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1cc1a_venta_depart...,corredor responsable: gabriel lopez - mat. de ...,suites en condo hotel neuquen soho-i en nqn cap,https://thumbs4.properati.com/3/u6lh_t4t4kpe-9...,0,0,0,0,1,0,0,0,0,0,0,0,1
28637,28637,sell,apartment,posadas,|argentina|misiones|posadas|,argentina,misiones,3429886.0,NaN,NaN,NaN,3090000.0,ars,3057766.44,173298.56,103.0,NaN,1682.510291,NaN,NaN,NaN,http://www.properati.com.ar/17wul_venta_depart...,"(de23) vende dptos impecables, terminacion mod...",vendo depto de primera calidad a metros del mi...,https://thumbs4.properati.com/7/b0iycxdkf7xvrh...,4,1,0,1,1,1,0,0,0,0,0,0,1
94492,94492,sell,apartment,olivos,|argentina|bs.as. g.b.a. zona norte|vicente ló...,argentina,bs.as. g.b.a. zona norte,3430310.0,NaN,NaN,NaN,240000.0,usd,4234680.00,240000.00,NaN,75.0,NaN,3200.000000,NaN,NaN,http://www.properati.com.ar/1bfmw_venta_depart...,semipiso en pozo. 2 dormitorios (principal en ...,edificio a estrenar | punto norte | felix de a...,https://thumbs4.properati.com/6/q2tcq_cmy4exl1...,0,0,0,0,0,0,1,0,0,0,0,0,1


In [250]:
list(en_pozo.description)

['inversión emprendimiento en pozo. apart hotel .los departamentos cuentan con un ambiente amplio y luminoso con pisos de porcelanato pulido, cocina integrada con muebles bajo y sobre mesada de melanina con acero inoxidable, mesada de mármol negro. horno y anafes eléctricos. baño completo con artefactos ferrum y griferías fv. aberturas de aluminio anodizado y calefacción por aire acondicionado f/c.cocheras cubiertas disponibles. amenities: jacuzzi, sum, parrilla, y solárium. forma de financiación: 40% de anticipo + 24 cuotas en pesos ajustadas según cac (cámara argentina de la construcción)fecha de entrega: enero 2020 -  - nexo propiedades',
 'venta departamento 3 ambientes en villa luro, en pozo.excelentes departamentos de categoría, ubicados desde el primero al octavo piso, frente y contrafrente.edificiocontrol acceso de entrada al edificio y ascensor.control acceso en cochera.cocheras cubiertas y descubiertas.ascensores.iluminación en espacios comunes.grupo electrógeno para servicio

<h5>Después de varios samples, confirmo que el regex coincide con inmuebles que son en pozo. Dado que son el 1,5% de los datos y que no conforman bases de datos actuales (porque son a construir), entendemos que no son óptimos para entrenar a un modelo que después vaya a predecir.
Por eso, dropeamos los inmuebles en pozo

In [251]:
cantidad_anterior = data.shape[0]
cantidad_anterior

120278

In [252]:
data = data.loc[data['en_pozo_regex'] == 0,:]
data.en_pozo_regex.value_counts()

0    118495
Name: en_pozo_regex, dtype: int64

In [253]:
(cantidad_anterior - data.shape[0]) / cantidad_anterior

0.014823991087314388

In [254]:
data.shape[0]

118495

In [255]:
data.head(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex,en_pozo_regex
0,0,sell,ph,mataderos,|argentina|capital federal|mataderos|,argentina,capital federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,usd,1093959.0,62000.0,55.0,40.0,1127.272727,1550.0,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 ambientes tipo casa planta baja por pasillo,...",2 amb tipo casa sin expensas en pb,https://thumbs4.properati.com/8/bluuyihjlhgiik...,2,0,0,0,0,0,0,0,1,0,0,1,0
1,1,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|,argentina,bs.as. g.b.a. zona sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,usd,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,venta de departamento en décimo piso al frente...,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpvbu2ztha7jv...,0,0,1,0,0,0,1,0,1,1,0,0,0


In [256]:
# ahora dropeo la columna "en pozo" porque ya no tiene info útil
data = data.drop(columns = ['en_pozo_regex'])
data.head(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex
0,0,sell,ph,mataderos,|argentina|capital federal|mataderos|,argentina,capital federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,usd,1093959.0,62000.0,55.0,40.0,1127.272727,1550.0,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 ambientes tipo casa planta baja por pasillo,...",2 amb tipo casa sin expensas en pb,https://thumbs4.properati.com/8/bluuyihjlhgiik...,2,0,0,0,0,0,0,0,1,0,0,1
1,1,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|,argentina,bs.as. g.b.a. zona sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,usd,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,venta de departamento en décimo piso al frente...,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpvbu2ztha7jv...,0,0,1,0,0,0,1,0,1,1,0,0


#### 2.13) Presenta edificio + dropear

En algunas descripciones se nota que presentan un edificio completo cuyas unidades pueden tener distintas características (distinta cantidad de ambientes, entre otras). A continuación se hace un breve análisis de esto y se dropean estos casos, los cuales son muy pocos.

In [257]:
patron_presenta_edificio = "(?P<presenta_edificio>presenta\s?edificio)"
regex_presenta_edificio  = re.compile(patron_presenta_edificio)

In [258]:
columna_regex(regex_presenta_edificio, 'regex_presenta_edificio')

Esta es la cantidad neta: 
0    118411
1        84
Name: regex_presenta_edificio, dtype: int64

Esta es la cantidad en %: 
0    0.999291
1    0.000709
Name: regex_presenta_edificio, dtype: float64


In [259]:
aux_presenta_edificio = data[(data.regex_presenta_edificio == 1)].sample(5)
aux_presenta_edificio

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex,regex_presenta_edificio
117930,117930,sell,apartment,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,NaN,NaN,NaN,1538660.0,ars,1522609.36,86293.71,77.0,61.0,1120.697532,25223.934426,NaN,NaN,http://www.properati.com.ar/1ce5p_venta_depart...,"duit propiedades presenta edificio mapa 03, en...",departamento - providencia,https://thumbs4.properati.com/8/be5lzbdg9kofkz...,0,1,0,0,1,0,1,0,0,0,0,0,1
81873,81873,sell,apartment,san lorenzo,|argentina|santa fe|san lorenzo|,argentina,santa fe,3837090.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,65.0,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1ayis_venta_depart...,sergio villella inmobiliaria presenta edificio...,av. belgrano 2100 - venta departamento un dorm...,https://thumbs4.properati.com/0/s3vcmiv9kc9sq3...,0,0,1,0,0,0,1,0,0,0,0,0,1
81647,81647,sell,apartment,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,"-31.4160544,-64.1677818",-31.416054,-64.167782,NaN,NaN,NaN,NaN,55.0,43.0,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1ay9k_venta_depart...,sergio villella inmobiliaria presenta edificio...,departamento - general paz,https://thumbs4.properati.com/9/kjlbamzo_zi6g4...,0,0,0,0,0,0,1,0,0,0,0,0,1
81733,81733,sell,apartment,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1ayd9_venta_depart...,sergio villella inmobiliaria presenta edificio...,departamento - general paz,https://thumbs4.properati.com/6/fncbq5irdgzloc...,0,0,0,0,0,0,1,0,0,0,0,0,1
84214,84214,sell,apartment,rosario,|argentina|santa fe|rosario|,argentina,santa fe,3838574.0,NaN,NaN,NaN,1220000.0,ars,1207273.39,68422.08,40.0,35.0,1710.552000,34857.142857,NaN,NaN,http://www.properati.com.ar/1b1gv_venta_depart...,sergio villella inmobiliaria presenta edificio...,rodriguez 1100 - departamento monoambiente en ...,https://thumbs4.properati.com/0/ejvz4cywn3mb64...,1,0,1,0,0,0,1,0,0,0,0,0,1


In [260]:
list(aux_presenta_edificio.description)

['duit propiedades presenta edificio mapa 03, en un lugar estratégico de la ciudad de córdoba, con excelentes acceso a los más importantes centros comerciales.mapa 03 cuenta con departamentos de 1 y 2 dormitorios, en cuatros tipologias distintas.especificaciones técnicas:-pisos de porcelanato-carpintería de aluminio en grandes ventanales-cocina completamente equipada con muebles bajo mesada y alacenas-mesada de granito natural-sanitario de primer calidad con bañera-grifería de monocomando cromada en cocinadaedificio totalmente eléctrico con solárium y asadores en terraza.características departamento: 1 dormitorio, amplio living comedor con posibilidad dividir en segundo dormitorio, dos baños completos con bañera, cocina separada y balcón hacia la calle.',
 'sergio villella inmobiliaria presenta edificio carcaraes land ubicado en la ciudad de carcarañá, en la calle av. belgrano esquina laprida.este edificio cuenta con 11 pisos, 4 departamentos de 1 dormitorio con balcones al frente y al

In [261]:
cantidad_anterior = data.shape[0]
cantidad_anterior

118495

In [262]:
data = data.loc[data['regex_presenta_edificio'] == 0,:]
data.regex_presenta_edificio.value_counts()

0    118411
Name: regex_presenta_edificio, dtype: int64

In [263]:
(cantidad_anterior - data.shape[0]) / cantidad_anterior

0.0007088906704924258

In [264]:
cantidad_anterior - data.shape[0]

84

In [265]:
data.head(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex,regex_presenta_edificio
0,0,sell,ph,mataderos,|argentina|capital federal|mataderos|,argentina,capital federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,usd,1093959.0,62000.0,55.0,40.0,1127.272727,1550.0,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 ambientes tipo casa planta baja por pasillo,...",2 amb tipo casa sin expensas en pb,https://thumbs4.properati.com/8/bluuyihjlhgiik...,2,0,0,0,0,0,0,0,1,0,0,1,0
1,1,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|,argentina,bs.as. g.b.a. zona sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,usd,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,venta de departamento en décimo piso al frente...,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpvbu2ztha7jv...,0,0,1,0,0,0,1,0,1,1,0,0,0


In [266]:
# ahora dropeo la columna "regex_presenta_edificio" porque ya no tiene info útil
data = data.drop(columns = ['regex_presenta_edificio'])
data.head(2)	

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex
0,0,sell,ph,mataderos,|argentina|capital federal|mataderos|,argentina,capital federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,usd,1093959.0,62000.0,55.0,40.0,1127.272727,1550.0,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 ambientes tipo casa planta baja por pasillo,...",2 amb tipo casa sin expensas en pb,https://thumbs4.properati.com/8/bluuyihjlhgiik...,2,0,0,0,0,0,0,0,1,0,0,1
1,1,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|,argentina,bs.as. g.b.a. zona sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,usd,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,venta de departamento en décimo piso al frente...,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpvbu2ztha7jv...,0,0,1,0,0,0,1,0,1,1,0,0


<a id="3"></a>
## 3) Limpieza y transformación de número de piso

[Ir a índice](#indice)

In [267]:
data.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'expenses', 'properati_url', 'description', 'title', 'image_thumbnail',
       'ambientes_regex', 'parrilla_regex', 'cochera_regex', 'pileta_regex',
       'sum_regex', 'gym_regex', 'balcon_regex', 'jardin_regex',
       'lavadero_regex', 'estrenar_regex', 'quincho_regex', 'patio_regex'],
      dtype='object')

In [268]:
data.floor.value_counts()

1.0       2042
2.0       1515
3.0        893
4.0        616
5.0        474
6.0        473
7.0        346
8.0        244
9.0        199
10.0       178
11.0       120
12.0       113
13.0        65
14.0        46
15.0        37
17.0        28
16.0        22
18.0        15
22.0        15
19.0        10
27.0         6
25.0         6
20.0         6
21.0         6
108.0        5
1609.0       4
104.0        4
403.0        4
24.0         3
301.0        3
907.0        3
33.0         3
702.0        3
31.0         3
36.0         3
106.0        3
23.0         3
103.0        2
64.0         2
32.0         2
50.0         2
105.0        2
121.0        2
113.0        2
601.0        2
42.0         2
49.0         2
905.0        2
55.0         2
304.0        2
136.0        2
1104.0       2
1608.0       2
313.0        2
40.0         2
208.0        2
29.0         2
102.0        2
135.0        2
408.0        2
39.0         2
1504.0       2
109.0        2
908.0        2
524.0        2
704.0        2
505.0     

In [269]:
data.floor.notnull().sum()/len(data.floor)

0.06500240687098327

In [270]:
data.floor.isnull().sum()

110714

Altísima cantidad de nulos.

In [271]:
# chequeo sample de los valores no nulos de "floor", a ver qué me dicen sus descripciones vs el dato en sí
floor_notnull = data[data.floor.notnull()].sample(5)
floor_notnull

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex
19758,19758,sell,apartment,palermo,|argentina|capital federal|palermo|,argentina,capital federal,3430234.0,"-34.58522,-58.426652",-34.585220,-58.426652,114000.0,usd,2011473.0,114000.0,NaN,44.0,NaN,2590.909091,1.0,NaN,http://www.properati.com.ar/17amp_venta_depart...,corredor responsable: mónica silvia martinuzzi...,venta 2 amb en inmejorable zona de palermo soho!,https://thumbs4.properati.com/6/ao7g_pu0wbt7hd...,2,0,0,0,0,0,1,0,0,0,0,0
76589,76589,sell,apartment,flores,|argentina|capital federal|flores|,argentina,capital federal,3433918.0,"-34.6326795,-58.466863",-34.632680,-58.466863,150000.0,usd,2646675.0,150000.0,61.0,57.0,2459.016393,2631.578947,10.0,4500.0,http://www.properati.com.ar/1ars1_venta_depart...,departamento de 3 ambientes. frente. cochera m...,departamento en venta,https://thumbs4.properati.com/5/y3lutsck7cu8g_...,3,0,1,0,0,0,1,0,0,0,0,0
55793,55793,sell,apartment,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,"-31.3845315,-64.2634733",-31.384532,-64.263473,210000.0,usd,3705345.0,210000.0,NaN,90.0,NaN,2333.333333,1.0,NaN,http://www.properati.com.ar/19pn7_venta_depart...,departamento en venta de 2 dorm. en cordoba,mai tei depto en venta 2 dorm y 2 baños,https://thumbs4.properati.com/2/wso3p91pvjlgh_...,0,0,0,0,0,0,0,0,0,0,0,0
11902,11902,sell,apartment,palermo,|argentina|capital federal|palermo|,argentina,capital federal,3430234.0,"-34.5817925,-58.4494905",-34.581792,-58.449491,153000.0,usd,2699608.5,153000.0,NaN,NaN,NaN,NaN,4.0,NaN,http://www.properati.com.ar/16ia3_venta_depart...,venta departamento 2 ambientes palermounidad a...,departamento en venta,https://thumbs4.properati.com/9/8xkos4t8r3uxng...,2,1,1,0,0,0,0,0,1,1,0,0
110214,110214,sell,apartment,mar del plata,|argentina|buenos aires costa atlántica|mar de...,argentina,buenos aires costa atlántica,3430863.0,"-38.005944,-57.54519",-38.005944,-57.545190,69000.0,usd,1217470.5,69000.0,NaN,44.0,NaN,1568.181818,12.0,NaN,http://www.properati.com.ar/1c3cm_venta_depart...,corredor responsable: gilda clauso - cmdpdjmp ...,plaza colon 2 ambientes al frente muy luminoso!!,https://thumbs4.properati.com/9/rfbac1vcjtwqfy...,2,0,0,0,0,0,0,0,0,0,0,0


In [272]:
# veo los comentarios con todos sus caracteres
list(floor_notnull.description)

['corredor responsable: mónica silvia martinuzzi - cucicba 5172contacto: verónica zaltzman - mls id # 420621048-7departamento de 2 amb de 44,6 m2 en excelente ubicación en palermo soho. a 3 cuadras de plaza italia y 3 de juan b. justo!!! luminoso. muy buena distribución. hall de entrada, cocina independiente. lyc de 4x3.50. balcón al frente. dormitorio de 3,3 x 3,1 con amplio placard y ventana al frente. baño completo con ventilación exterior. hall íntimo con placard. pisos de parquet. paliers y zonas comunes (palier y escaleras) recién pintadas. sólida calidad constructiva.   situado en palermo soho, zona de diseño, arte y polo gastronómico, y a su vez muy cerca de todos los medios de transporte: líneas de colectivo, trenes y subte.  apto crédito.  expensas mayo: $2070.-  ',
 'departamento de 3 ambientes. frente. cochera movil. balcón. dormitorio principal en suite con vestidor. dormitorio secundario con placard.  calefacción y a/a por split. pisos de madera flotante. aberturas con ma

Después de correr varias veces el sample y comparar sus descripciones con el dato de floor, confirmo que, cuando figura, me indica el número de piso en el que se encuentra un depto.

In [273]:
# Ahora, podría confirmar si todos los inmuebles que tienen el dato de número de piso son departamentos
data[data.floor.notnull()].property_type.value_counts()

apartment    6326
house         762
ph            569
store          40
Name: property_type, dtype: int64

In [274]:
# chequeo sample de los valores no nulos de "floor" y house, a ver qué me dicen sus descripciones vs el dato en sí
floor_notnull_house = data[(data.floor.notnull()) & (data['property_type'] == 'house')].sample(5)
floor_notnull_house

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex
64093,64093,sell,house,ezeiza,|argentina|bs.as. g.b.a. zona sur|ezeiza|ezeiza|,argentina,bs.as. g.b.a. zona sur,3433975.0,"-34.852522,-58.510394",-34.852522,-58.510394,110000.0,usd,1940895.0,110000.0,NaN,53.0,NaN,2075.471698,2.0,NaN,http://www.properati.com.ar/1aa0u_venta_casa_e...,corredor responsable: martin manias - cmclz 40...,duplex apto crédito todos los bancos,https://thumbs4.properati.com/6/_cjbyoklrzpgum...,3,1,1,0,0,0,0,1,0,0,0,1
35406,35406,sell,house,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,"-31.3564644,-64.2112299",-31.356464,-64.211230,170000.0,usd,2999565.0,170000.0,NaN,143.0,NaN,1188.811189,2.0,NaN,http://www.properati.com.ar/18e9t_venta_casa_c...,casa en venta de 2 dorm. en cordoba,casa en venta 3 dorm nuevo poeta lugones,https://thumbs4.properati.com/6/l0dgxqha_wdatd...,0,0,0,0,0,0,0,0,0,0,0,0
9734,9734,sell,house,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,"-31.3631776,-64.2058457",-31.363178,-64.205846,240000.0,usd,4234680.0,240000.0,NaN,270.0,NaN,888.888889,2.0,NaN,http://www.properati.com.ar/165gh_venta_casa_c...,casa en venta de 3 dorm. en cordoba,casa venta poeta lugones 5 dorm,https://thumbs4.properati.com/4/mcvfqaqzehtrjy...,0,0,0,0,0,0,0,0,0,0,0,0
14197,14197,sell,house,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,"-31.3914504,-64.2546716",-31.391450,-64.254672,1950000.0,usd,34406775.0,1950000.0,NaN,150.0,NaN,13000.000000,1.0,NaN,http://www.properati.com.ar/16rcj_venta_casa_c...,casa en venta de 2 dorm. en cordoba,casa en venta de 2 dorm. en cordoba,https://thumbs4.properati.com/1/trqlynv8rpdwww...,0,0,0,0,0,0,0,0,0,0,0,0
119250,119250,sell,house,quilmes,|argentina|bs.as. g.b.a. zona sur|quilmes|quil...,argentina,bs.as. g.b.a. zona sur,3429651.0,"-34.721417,-58.275547",-34.721417,-58.275547,280000.0,usd,4940460.0,280000.0,NaN,190.0,NaN,1473.684211,1.0,NaN,http://www.properati.com.ar/1cge8_venta_casa_q...,corredor responsable: martin veltri - c.m.q 80...,venta de casa 4 ambientes en quilmes oeste,https://thumbs4.properati.com/7/ohbhq8ammsswim...,4,1,0,1,0,0,1,1,1,0,0,1


In [275]:
# veo los comentarios con todos sus caracteres
list(floor_notnull_house.description)

['corredor responsable: martin manias - cmclz 4037contacto: gustavo gurskis - mls id # 420541035-107duplex apto crédito  3 ambientes frente al country allá en el sur   entrada con portón corredizo y cochera. patio al frente de 6 x 6 mts  living, comedor y cocina integrada de 3 x 3,30 mts, con muebles bajo mesada y pisos de porcelanato pulido.  dos dormitorios con placard y t/b. baño de 1,5 x 2 mts con puerta corrediza.  jardín al fondo con parrilla.  techo de tejas y losa.  pavimeno a 20 mts de la entrada. servicios de luz, gas y agua de pozo.   zona residencial-  ',
 'casa en venta de 2 dorm. en cordoba',
 'casa en venta de 3 dorm. en cordoba',
 'casa en venta de 2 dorm. en cordoba',
 'corredor responsable: martin veltri - c.m.q 805contacto: gisel scutti - mls id # 420171085-101hermoso chalet 4 ambientes en venta en quilmes oeste sobre lote de 8,66 x 47. jardín al frente y garaje cubierto para un auto. en planta baja: entrada a gran living en desnivel, amplio comedor, baño, cocina.  e

Después de correr varias veces el sample y comparar sus descripciones con el dato de floor únicamente para cuando la propiedad es "house", parece ser que el dato no me indica nada fiable (a veces aparece piso 1, a veces aparece "2" y en la descripción dice "2 dormitorios", etc.).

In [276]:
# chequeo sample de los valores no nulos de "floor" y house, a ver qué me dicen sus descripciones vs el dato en sí
floor_notnull_ph = data[(data.floor.notnull()) & (data['property_type'] == 'ph')].sample(5)
floor_notnull_ph

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex
5357,5357,sell,ph,abasto,|argentina|capital federal|abasto|,argentina,capital federal,NaN,"-34.6065692,-58.4111367",-34.606569,-58.411137,160000.0,usd,2823120.00,160000.00,160.0,63.0,1000.000000,2539.682540,1.0,NaN,http://www.properati.com.ar/15pyr_venta_ph_aba...,"4 ambientes con patio y terraza al frente, en ...",ph en venta,https://thumbs4.properati.com/6/ncpcfv0nr7d1bg...,4,0,0,0,0,0,1,0,0,0,0,1
29871,29871,sell,ph,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,NaN,NaN,NaN,1100000.0,ars,1088525.19,61692.04,54.0,44.0,1142.445185,25000.000000,4.0,NaN,http://www.properati.com.ar/18006_venta_ph_cor...,las americas - venta - departamento - 1 dormi...,ph en venta,https://thumbs4.properati.com/5/o1tbrtqzr9zwsh...,0,1,1,0,1,0,1,0,0,0,0,0
29797,29797,sell,ph,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,55.0,NaN,NaN,5.0,NaN,http://www.properati.com.ar/17zxz_venta_ph_cor...,cenit 5 - venta - departamento - 1 dormitorio ...,ph en venta,https://thumbs4.properati.com/7/pmi-kw2kfy2nus...,0,0,1,1,1,1,1,0,0,0,0,0
29790,29790,sell,ph,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,86.0,NaN,NaN,2.0,NaN,http://www.properati.com.ar/17zxs_venta_ph_cor...,cenit 5 - venta - departamento - 2 dormitorio ...,ph en venta,https://thumbs4.properati.com/9/qhjxzre-vx92na...,0,0,1,1,1,1,1,0,0,0,0,0
49767,49767,sell,ph,neuquén,|argentina|neuquén|neuquén|,argentina,neuquén,3843123.0,"-38.9513389666,-68.0664865562",-38.951339,-68.066487,2229000.0,ars,2205747.94,125010.51,48.0,46.0,2604.385625,48456.521739,5.0,NaN,http://www.properati.com.ar/19css_venta_ph_neu...,"venta de departamento 1 dormitorio en centro, ...",ph en venta,https://thumbs4.properati.com/3/b7skfe1y_d9jrw...,1,0,1,0,1,0,1,0,0,0,0,0


In [277]:
# veo los comentarios con todos sus caracteres
list(floor_notnull_ph.description)

['4 ambientes con patio y terraza al frente, en venta, en abasto!!63m2 cubiertos + 97m2 descubiertos(63m2 cubiertos + 13m2 descubiertos por escritura)primer piso por escalera, living comedor, 3 dormitorios con pisos de parquet y salida al balcón corrido de 6 x 1, baño completo, cocina completa con salida al patio de 7m2 y acceso a gran terraza de 84m2 con cuarto de herramientas.cañerias de gas hechas a nuevo y tanque de agua propio!sucursal v. crespo ()sucursal almagro () deja constancia que los m2 indicados son aproximados, al igual que las medidas parciales de los ambientes, y el valor consignado de expensas mensuales está sujeto a verificación y/o ajustes"fotos de carácter no contractual -  - ebertolotti',
 'las americas - venta  - departamento - 1 dormitorio - 2 baños - nueva córdoba - cordoba* ubicado : a solo una cuadra de ciudad universitaria, y de la plaza las américas. * seguridad:  cámaras de seguridad en ingresos y espacios comunes* hall ingreso: jerarquia.* cochera: en subs

La situación de los PH parece similar a las de departamentos

In [278]:
# chequeo sample de los valores no nulos de "floor" y store, a ver qué me dicen sus descripciones vs el dato en sí
floor_notnull_store = data[(data.floor.notnull()) & (data['property_type'] == 'store')].sample(5)
floor_notnull_store

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex
109595,109595,sell,store,tandil,|argentina|buenos aires interior|tandil|tandil|,argentina,buenos aires interior,3427832.0,"-37.334242,-59.134316",-37.334242,-59.134316,270000.0,usd,4764015.0,270000.0,NaN,20.0,NaN,13500.000000,2.0,NaN,http://www.properati.com.ar/1c2e6_venta_local_...,corredor responsable: mirta leguizamon - cmcpd...,excelente local comercial/ oficinas,https://thumbs4.properati.com/8/apn09v65k4wmuw...,0,0,1,0,0,0,0,0,0,0,0,0
119511,119511,sell,store,córdoba,|argentina|córdoba|córdoba|,argentina,córdoba,3860259.0,"-31.3640558,-64.1745196",-31.364056,-64.174520,NaN,NaN,NaN,NaN,NaN,260.0,NaN,NaN,2.0,NaN,http://www.properati.com.ar/1cgr4_venta_local_...,galpon en venta de 2 dorm. en cordoba,venta - galpón con oficina - escritura -,https://thumbs4.properati.com/8/hdhdptutz966ww...,0,0,0,0,0,0,0,0,0,0,0,0
2699,2699,sell,store,mar de ajó,|argentina|buenos aires costa atlántica|mar de...,argentina,buenos aires costa atlántica,3430866.0,"-36.72141266,-56.67832184",-36.721413,-56.678322,NaN,NaN,NaN,NaN,18.0,18.0,NaN,NaN,1.0,NaN,http://www.properati.com.ar/15kfr_venta_local_...,muy buen local comercial con ubicación privile...,u$d 45.000 - local en venta - avenida libertad...,https://thumbs4.properati.com/6/lngej9rcqjsyju...,0,1,0,0,0,0,0,0,0,0,0,0
23557,23557,sell,store,villa rosa,|argentina|bs.as. g.b.a. zona norte|pilar|vill...,argentina,bs.as. g.b.a. zona norte,3427377.0,NaN,NaN,NaN,130000.0,usd,2293785.0,130000.0,130.0,130.0,1000.0,1000.000000,1.0,NaN,http://www.properati.com.ar/17m30_venta_local_...,"galerã­a la escala, caamaã±o 1175, a 100 metro...","local 130m² con 2 plantas, pilar, villa rosa, ...",https://thumbs4.properati.com/5/y5i6xvd71w2ejk...,0,0,0,0,0,0,0,0,0,0,0,0
34996,34996,sell,store,san fernando,|argentina|bs.as. g.b.a. zona norte|san fernando|,argentina,bs.as. g.b.a. zona norte,3429088.0,NaN,NaN,NaN,40000.0,usd,705780.0,40000.0,NaN,30.0,NaN,1333.333333,1.0,NaN,http://www.properati.com.ar/18d89_venta_local_...,venta en block o x unidades local 3 x 8 c-baão...,"local en yrigoyen, av. hipólito 2300, bs.as. g...",https://thumbs4.properati.com/2/x1qyotxneananx...,0,0,0,0,0,0,0,0,0,0,0,0


In [279]:
# veo los comentarios con todos sus caracteres
list(floor_notnull_store.description)

['corredor responsable: mirta leguizamon - cmcpdja 1610contacto: geraldina rebello - mls id # 420821011-6excelente local de 180m2 de frente a contrafrente, 2do piso. adaptable a todo tipo de oficinas. totalmente iluminadas, goza de 4 kitchenette y 4 baños, ascensor, dos cocheras inherentes a ellas, además que posee cómodo estacionamiento. ubicado en una de las  arterias principales de la ciudad, a 4 cuadras del centro comercial y paseos turísticos. construcciones innovadoras y funcionales en una ubicación inmejorable. importante financiaciòn.',
 'galpon en venta de 2 dorm. en cordoba',
 'muy buen local comercial con ubicación privilegiada - 2,70 m de vidriera - a 4 cuadras de avenida costanera y mar, 3 cuadras de peatonal irigoyen, 2 cuadras cotosobre la avenida principal de acceso a la ciudad y en el centro, zona comercial de alto tránsito vehicular y peatonalvalor: u$s 45.000,-visitas desde la costa: 02257-15-414737  011-15-56094072  desde caba: 011- 011-ricardo fernández asesoría in

Pareciera ser una situación similar a la de departamentos.

Hay que generar un regex para ver si logramos menor cantidad de nulos

In [280]:
patron_piso = "(?P<piso>\d?\d.?º?.?.?piso)"
regex_piso  = re.compile(patron_piso)

In [281]:
columna_regex(regex_piso, 'regex_piso')

Esta es la cantidad neta: 
0    104794
1     13617
Name: regex_piso, dtype: int64

Esta es la cantidad en %: 
0    0.885002
1    0.114998
Name: regex_piso, dtype: float64


In [282]:
aux_floor = data[(data.regex_piso == 1)].sample(5)
aux_floor

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex,regex_piso
27300,27300,sell,apartment,almagro,|argentina|capital federal|almagro|,argentina,capital federal,3436397.0,"-34.601773,-58.419141",-34.601773,-58.419141,103000.0,usd,1817383.5,103000.0,NaN,45.0,NaN,2288.888889,4.0,NaN,http://www.properati.com.ar/17t4z_venta_depart...,corredor responsable: jorge salafia - cucicba ...,fideicomiso en almagro entrega mayo 2018,https://thumbs4.properati.com/1/te8xi9dw3no8jx...,1,1,1,0,1,0,1,0,0,0,0,0,1
39485,39485,sell,apartment,ciudad jardín lomas del palomar,|argentina|bs.as. g.b.a. zona oeste|tres de fe...,argentina,bs.as. g.b.a. zona oeste,NaN,"-34.5984128,-58.5915638",-34.598413,-58.591564,71000.0,usd,1252759.5,71000.0,44.0,44.0,1613.636364,1613.636364,2.0,900.0,http://www.properati.com.ar/18ofu_venta_depart...,roldan propiedades - venta de departamento 2 a...,departamento en venta,https://thumbs4.properati.com/5/naycz6-sqafwp9...,2,0,0,0,0,0,1,1,0,0,0,0,1
117623,117623,sell,apartment,belgrano,|argentina|capital federal|belgrano|,argentina,capital federal,3436077.0,NaN,NaN,NaN,187000.0,usd,3299521.5,187000.0,53.0,40.0,3528.301887,4675.000000,NaN,NaN,http://www.properati.com.ar/1cdri_venta_depart...,en una ubicación única...,departamento - belgrano,https://thumbs4.properati.com/1/zshrxqrpjzz5yw...,1,1,0,1,1,0,0,0,0,0,0,1,1
65746,65746,sell,apartment,córdoba,|argentina|córdoba|,argentina,córdoba,3860255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1actp_venta_depart...,te invitamos a conocergran depsal iiien el cor...,departamento en nueva cordoba,https://thumbs4.properati.com/5/unqaylxusej7tc...,0,0,0,0,0,0,1,0,0,0,0,0,1
4666,4666,sell,apartment,rosario,|argentina|santa fe|rosario|,argentina,santa fe,3838574.0,"-32.9267159,-60.6647543",-32.926716,-60.664754,NaN,NaN,NaN,NaN,67.0,67.0,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15nee_venta_depart...,"edificio de alta categoría, con inigualable vi...",torreon del rio,https://thumbs4.properati.com/6/zrmm9y8fjkr7gc...,0,0,1,1,1,1,1,1,1,0,1,1,1


In [283]:
list(aux_floor.description)

['corredor responsable: jorge salafia - cucicba 5666contacto: marta peña - mls id # 420211006-163emprendimiento en construcción de 9 pisos, 5 unidades por piso, con cocheras en pb y subsuelo,  y solarium y parrilla comunes en azotea. entrega mayo 2018.  esta unidad es un monoambiente al frente con cocina integrada,  baño completo, y balcón aterrazado. canalización para aire acondicionado, teléfono y video.  muy alto nivel de calidad de terminaciones. sobre una calle arbolada, con ubicación estratégica a 2 cuadras de av. córdoba, y a 2 cuadras de av. corrientes y del subte línea b estación medrano. y cercana a universidad de palermo, universidad tecnológica nacional, shopping abasto y sanatorio guemes.  ',
 'roldan propiedades - venta de departamento 2 ambientes en ciudad jardín 2° piso al frente, cocina, living comedor con balcon, dormitorio, baño completo. todo a nuevo. valor de venta u$s71.000.- libre de gastos - oficina inmobiliaria - boulevard san martin 3102 - el palomar - 4758694

<h5>Entre los samples y probar distintos intentos de regex, no se llega a mejor cantidad de datos (con calidad). En las descripciones no parece ser habitual detallar esa característica (aunque los hay, no es significativa la cantidad). Por tal motivo, dropeamos la columna

In [284]:
data = data.drop(columns = ['floor', 'regex_piso'])
data.head(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,expenses,properati_url,description,title,image_thumbnail,ambientes_regex,parrilla_regex,cochera_regex,pileta_regex,sum_regex,gym_regex,balcon_regex,jardin_regex,lavadero_regex,estrenar_regex,quincho_regex,patio_regex
0,0,sell,ph,mataderos,|argentina|capital federal|mataderos|,argentina,capital federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,usd,1093959.0,62000.0,55.0,40.0,1127.272727,1550.0,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 ambientes tipo casa planta baja por pasillo,...",2 amb tipo casa sin expensas en pb,https://thumbs4.properati.com/8/bluuyihjlhgiik...,2,0,0,0,0,0,0,0,1,0,0,1
1,1,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|,argentina,bs.as. g.b.a. zona sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,usd,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,venta de departamento en décimo piso al frente...,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpvbu2ztha7jv...,0,0,1,0,0,0,1,0,1,1,0,0
